In [1]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from geopy.geocoders import Nominatim
import mysql.connector
from sqlalchemy import create_engine
import datetime

In [2]:
def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [3]:
df1 = pd.read_parquet('Naukri_raw1.parquet')
df2 = pd.read_parquet('Naukri_raw2.parquet')
df3 = pd.read_parquet('Naukri_raw3.parquet')

In [4]:
for i in range(1,4):
    try:
        os.remove(f'Naukri_raw{i}.parquetbak.parquet')
    except:
        None

In [5]:
df = pd.concat([df1,df2,df3],ignore_index=True)

In [6]:
df.columns = df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()

In [7]:
df = df[~df.isna().all(axis=1)]

In [8]:
del df['']

In [9]:
def fillsal(row):
    if 'not disclosed' in row['salary'].lower():
        retval = '-'.join(row['sal_range'].split('to'))
        return retval
    else:
        return row['salary']
df['salary'] = df[['salary','sal_range']].apply(fillsal,axis=1)

In [10]:
def extract_and_calculate_average(salary_str):
    l = []
    salary_str = salary_str.replace('Less than','')
    
    if '/month' in salary_str:
        salary_str = salary_str.replace('/month','').replace(',','')
        return (float(salary_str)*12)/100000
    
    try:
        a = salary_str.split('-')
        if a[0] == a[1]:
            salary_str = a[0]
    except:
        None
        
    try:    
        l = list(salary_str)
        if 'L' in l:    
            ind = l.index('L')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',','') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                l = [float(i)/100000 if len(i) > 4 else float(i) for i in l]
                l = sum(l)/2
                return l
            else:
                print(l,'****',salary_str,end='****')
                l = (float(l)/100000 if len(l) > 4 else float(l))
                print(l)
                return l
        
        elif 'C' in l:
            ind = l.index('C')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',','') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                l = [float(i)*100 if len(i) > 4 else float(i) for i in l]
                l = sum(l)/2
                return l
            else:
                l = (float(l)/100 if len(l) > 4 else float(l))
                return l[0]*100
        
        elif 'P' in l:
            ind = l.index('P')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',','') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:
                l = l.split('-')
                l = [float(i)/100000 if len(i) > 4 else float(i) for i in l]
                l = sum(l)/2
                return l 
            else:
                l = (float(l)/100000 if len(l) > 4 else float(l))
                return l 
    except:
        return salary_str
df['saledit'] = df['salary'].apply(extract_and_calculate_average)


3 **** 3 Lacs P.A.****3.0
3 **** 3 Lacs P.A.****3.0
5 **** 5 Lacs P.A.****5.0
3 **** 3 Lacs P.A.****3.0
5 **** 5 Lacs P.A.****5.0
3 **** 3 Lacs P.A.****3.0
4 **** 4 Lacs P.A.****4.0
4 **** 4 Lacs P.A.****4.0
3 **** 3 Lacs P.A.****3.0
12 **** 12 Lacs P.A.****12.0
19 **** 19 Lacs P.A. (Including Variable: 12%)****19.0
45 **** 45 Lacs-1 Cr P.A.****45.0
25 **** 25 Lacs P.A.****25.0
40 **** 40 Lacs-1 Cr P.A.****40.0
30 **** 30 Lacs P.A.****30.0
40 **** 40 Lacs P.A.****40.0
40 **** 40 Lacs P.A.****40.0
40 **** 40 Lacs-1 Cr P.A.****40.0
40 **** 40 Lacs P.A.****40.0
40 **** 40 Lacs P.A.****40.0
35 **** 35 Lacs-1 Cr P.A.****35.0
35 **** 35 Lacs-1 Cr P.A.****35.0
35 **** 35 Lacs P.A.****35.0
25 **** 25 Lacs P.A.****25.0
27.5 **** 27.5 Lacs P.A.****27.5
25 **** 25 Lacs P.A.****25.0
40 **** 40 Lacs P.A.****40.0
30 **** 30 Lacs P.A.****30.0
30 **** 30 Lacs P.A.****30.0
30 **** 30 Lacs P.A.****30.0
40 **** 40 Lacs-1 Cr P.A.****40.0
35 **** 35 Lacs-1 Cr P.A.****35.0
40 **** 40 Lacs-1 Cr P.A.****40.0


In [11]:
df

,job_title,company_name,experience,salary,location,description,posted_on,current_date,link,sal_range,...,skill32,skill33,skill34,skill35,skill36,skill37,skill38,skill39,skill40,saledit
0,Pharmacist,Fortis,0 - 3 years,3-6,"Kolkata, Mumbai, New Delhi, Hyderabad, Pune, C...",Dispense prescription medications accurately a...,6 days ago,2024-04-02,https://www.naukri.com/job-listings-pharmacist...,3to6,...,None,None,None,None,None,None,None,None,None,None
1,Pharmacist,Fortis,0 - 3 years,3-6,"Kolkata, Mumbai, New Delhi, Hyderabad, Pune, C...",Dispense prescription medications accurately a...,6 days ago,2024-04-02,https://www.naukri.com/job-listings-pharmacist...,3to6,...,None,None,None,None,None,None,None,None,None,None
2,Software Engineer,Ford,5 - 7 years,3-6,"Kolkata, Mumbai, New Delhi, Hyderabad, Pune, C...",Software Engineer position responsible for dev...,4 days ago,2024-04-02,https://www.naukri.com/job-listings-software-e...,3to6,...,None,None,None,None,None,None,None,None,None,None
3,Dairy Feed Sales Officer,Cargill,2 - 6 years,3-6,"Kolkata, Mumbai, New Delhi, Hyderabad, Pune, C...",The Dairy Feed Sales Officer will be aligned t...,5 days ago,2024-04-02,https://www.naukri.com/job-listings-dairy-feed...,3to6,...,None,None,None,None,None,None,None,None,None,None
4,BOAT APPRENTICE TRAINEE,Nokia,0 - 1 years,3-6,"Kolkata, Mumbai, New Delhi, Hyderabad, Pune, C...",Assisting experienced employees with their dai...,13 days ago,2024-04-02,https://www.naukri.com/job-listings-boat-appre...,3to6,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64215,"Associate, Client Onboarding / Transitions / C...",BNY Mellon,1 - 4 years,0-3,Pune,"Under moderate direction, onboard clients/acco...",30+ days ago,2024-04-04,https://www.naukri.com/job-listings-associate-...,0to3,...,None,None,None,None,None,None,None,None,None,None
64216,"Associate Professional, Presale Support",Ingram Micro,2 - 5 years,0-3,Pune,"Supporting Actions:\nAnswer emails, calls and/...",30+ days ago,2024-04-04,https://www.naukri.com/job-listings-associate-...,0to3,...,None,None,None,None,None,None,None,None,None,None
64217,"Associate Professional, Presale Support",Ingram Micro,2 - 5 years,0-3,Pune,"Supporting Actions:\nAnswer emails, calls and/...",30+ days ago,2024-04-04,https://www.naukri.com/job-listings-associate-...,0to3,...,None,None,None,None,None,None,None,None,None,None
64218,"Associate Professional, Presale Support",Ingram Micro,2 - 5 years,0-3,Pune,"Supporting Actions:\nAnswer emails, calls and/...",30+ days ago,2024-04-04,https://www.naukri.com/job-listings-associate-...,0to3,...,None,None,None,None,None,None,None,None,None,None


In [12]:
ck = ['saledit','salary']

In [13]:
df[df['saledit'].apply(lambda x: isinstance(x, str))][ck]

,saledit,salary
317,₹ 2-3 Lacs P.A.,₹ 2-3 Lacs P.A.
325,₹ 3.25-4.25 Lacs P.A.,₹ 3.25-4.25 Lacs P.A.
427,₹ 3.5-5.5 Lacs P.A.,₹ 3.5-5.5 Lacs P.A.
1203,₹ 4-9 Lacs P.A.,₹ 4-9 Lacs P.A.
1288,₹ 5-15 Lacs P.A.,₹ 5-15 Lacs P.A.
...,...,...
63429,₹ 1.25-3 Lacs P.A.,₹ 1.25-3 Lacs P.A.
63658,₹ 1.75-3 Lacs P.A.,₹ 1.75-3 Lacs P.A.
64065,"₹ 50,000-80,000 P.A.","₹ 50,000-80,000 P.A."
64073,₹ 1.5-2.25 Lacs P.A.,₹ 1.5-2.25 Lacs P.A.


In [14]:
df['saledit'] = df['saledit'].fillna(0)

In [15]:
df = df[~df['saledit'].apply(lambda x: isinstance(x, str))]
df = df[~df['saledit'].apply(lambda x: x>100)]

In [16]:
df[df['saledit'].isna()][ck]

,saledit,salary


# Salary done

In [17]:
lidf = pd.read_excel('Linkedin_cleaned_latest_mapped_cols.xlsx')

c:\Users\PM\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [18]:
sdf = pd.DataFrame()
org_cols = ([col for col in lidf.columns if 'org' in col])
sdf['all_org_cols'] = lidf[org_cols].apply(lambda row: '$&'.join(row.dropna().astype(str)), axis=1)
sdf['all_org_cols']

companies = []
for i in sdf['all_org_cols']:
    for j in i.split('$&'):
        companies.append(j)
print(len(companies))

18


In [19]:
companiesdf = pd.DataFrame(companies,columns=['comps'])
companiesdf.drop_duplicates(inplace=True)
companiesdf.reset_index(drop = True, inplace=True)

In [20]:
companiesdf['comps'] = companiesdf['comps'].str.lower()
companiesdf['comps'] = companiesdf['comps'].str.replace(' · self-employed','').str.replace('· part-time','').str.replace('· freelance','')
companiesdf['comps'] = companiesdf['comps'].str.replace(' · full-time','').str.replace('  ',' ').str.replace('india','').str.replace('chennai','').str.replace('· internship','')
companiesdf['comps'] = companiesdf['comps'].str.strip()
companiesdf.drop_duplicates(inplace=True)
companiesdf.reset_index(drop = True, inplace=True)
companiesdf

,comps
0,


In [21]:
df['company_name'] = df['company_name'].str.lower()

In [22]:
len(df['company_name'].unique())

6973

In [23]:
# import pandas as pd
# from fuzzywuzzy import fuzz
# from multiprocessing import Pool
# from tqdm import tqdm  # Import tqdm for the progress bar

# # Function to calculate similarity scores and store them in a dictionary
# def calculate_similarity_scores(value, name_list, threshold=95):
#     best_score = 0
#     best_name = None

#     for name in name_list:
#         total_score = 0
#         for method in ['partial_ratio', 'token_set_ratio']:
#             similarity_score = getattr(fuzz, method)(value, name)
#             if similarity_score >= threshold:
#                 total_score += similarity_score

#         if total_score > best_score:
#             best_score = total_score
#             best_name = name

#     return best_name

# df['company_name'] = df['company_name'].astype(str)

# prl = companiesdf['comps'].tolist()

# tqdm.pandas()
# pool = Pool(processes=8)  # You can specify the number of processes here

# threshold = 95
# df['similarity_scores'] = df['company_name'].progress_apply(calculate_similarity_scores, args=(prl, threshold))

# pool.close()
# pool.join()

In [24]:
# df.to_excel('salary_and_company_mapping.xlsx',index= False)

In [25]:
# df = pd.read_excel('salary_and_company_mapping.xlsx')

In [26]:
# 'ratio'
# 'partial_ratio'
# 'token_set_ratio'
# 'token_sort_ratio'
# 'WRatio'
# 'SequenceMatcher'

In [27]:
df.rename(columns={'saledit':'mapped_average_sal'},inplace=True)
# df.rename(columns={'similarity_scores':'mapped_company_name'},inplace=True)

In [28]:
# df.to_excel('salary_and_company_mapping.xlsx',index= False)

# company name done

In [29]:
# df['avg_experience'] = df['experience'].str.replace('Yrs','')
# df['avg_experience'] = df['avg_experience'].str.split('-')

In [30]:
# def to_avg_exp(value):
#     if isinstance(value, list) and len(value) >= 2:
#         value[0] = int(value[0])
#         value[1] = int(value[1])
#         return sum(value) / 2
#     else:
#         # Handle cases where value is not a valid list
#         return value

# df['avg_experience'] = df['avg_experience'].apply(to_avg_exp)
# df['avg_experience']

# average age done

In [31]:
# def replacefun(value):
#     fun_l = ['ing','ca','ge','ltd.',')','is','private','the','ari','stem','-','fi','karnataka','bangalore','.com','one','h','ies','.','a','bengaluru, karnataka',
#              'business','actor','dec','pvt ltd','care']
#     if value in fun_l:
#         return None
#     else:
#         return value
# df['mapped_company_name'] = df['mapped_company_name'].apply(replacefun)

In [32]:
# for k,v in df['mapped_company_name'].value_counts().items():
#     print(k,v)

In [33]:
# len(df['mapped_company_name'].unique()) # 1855

In [34]:
# df.to_excel('salary_and_company_mapping.xlsx',index= False)

In [35]:
# df['mapped_company_name'].notnull().sum()

In [36]:
# df['company_name'].notnull().sum()

# mapping instute and size.

In [37]:
# ([col for col in lidf.columns if 'emp' in col])

In [38]:
# org_cols

In [39]:
# org_cols = ([col for col in lidf.columns if ('org' in col) and ('industry' not in col)])
# orglen = int(org_cols[-1][4:6])

# def splitjoin(value):
#     value = value.split(',')
#     if value[0] != '':
#         return value[0]
#     elif value[1] != '':
#         return value[1]
#     elif (value[0] == '') and (value[1] == ''):
#         return None 

# editdf=pd.DataFrame()
# for i in range(orglen+1):
#     subcol = [col for col in lidf.columns if 'org_'+str(i)+'_' in col]
#     llis1 = lidf['org_'+str(i)]
#     llis2 = lidf[subcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)
    
#     llis1 = llis1.fillna('').astype(str)
#     llis2 = llis2.fillna('').astype(str)
    
#     editdf['org_'+str(i)] = llis1+','+llis2
#     editdf['org_'+str(i)] = editdf['org_'+str(i)].apply(splitjoin)

In [40]:
# for col in editdf:
#     editdf[col] = editdf[col].str.lower()
#     editdf[col] = editdf[col].str.replace(' · self-employed','').str.replace('· part-time','').str.replace('· freelance','')
#     editdf[col] = editdf[col].str.replace(' · full-time','').str.replace('  ',' ').str.replace('india','').str.replace('chennai','').str.replace('· internship','')
#     editdf[col] = editdf[col].str.strip()


In [41]:
# df['mapped_company_name']

In [42]:
# for col in editdf:
#     print(col.split('_')[1])
# lidf.columns

In [43]:
# # def map_companies(ind, col):
# #     value = editdf[col][ind]
# #     colno = col.split('_')[1]
# #     if value == None:
# #         return
# #     prl = df['mapped_company_name'].tolist()
# #     for i in range(len(prl)):
# #         if prl[i] == value:
# #             df['comp_industry'][i] = lidf['company_'+'colno'+'_industry'][ind]
# # for col in editdf:
# #     for ind in range(len(editdf[col])):
# #         map_companies(ind, col)
    
# def map_companies(row):
#     index = row.name
#     for col in editdf.columns:
#         value = row[col]
#         if value is not None:
#             colno = col.split('_')[1]
            
#             if 'comp_industry' not in df.columns:
#                 df['comp_industry'] = None
            
#             if 'comp_industry' not in df.columns:
#                 df['comp_emp_count'] = None
            
#             matching_indices = df.index[df['mapped_company_name'] == value]
#             for idx in matching_indices:
#                 df.at[idx, 'comp_industry'] = lidf.at[index, f'company_{colno}_industry']
#                 df.at[idx, 'comp_emp_count'] = lidf.at[index, f'company_{colno}_emp_count']
# # Apply the mapping function to each row in editdf
# editdf.apply(map_companies, axis=1)

In [44]:
# df.to_excel('salary_and_company_mapping.xlsx',index = False)

In [45]:
# Check dtype
print(df['location'].dtype)

# Convert to string and apply operations
df['location'] = df['location'].astype(str).str.lower().str.replace('hybrid -', '').str.strip()

object


In [46]:
locset = set()
for loc in df['location']:
    sublocs = loc.split(',')
    for subloc in sublocs:
        locset.add(subloc.strip())

In [47]:
len(locset)

2386

In [48]:
adressdf = pd.read_csv('latlon/adresses.csv')

In [49]:
import ast
def str_to_dict(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return {} 

In [50]:
adressdf['adress'] = adressdf['adress'].apply(str_to_dict)

In [51]:
def get_city(adress):
    try:
        if adress['country'] == 'India':
            l = list(adress.keys())
            if 'city' in l: 
                return adress['city']
            # elif 'state_district' in l:
            #     return adress['state_district']
            elif 'district' in l:
                return adress['district']
            else:
                return None 
    except:
        return None 
adressdf['city'] = adressdf['adress'].apply(get_city)

In [52]:
mallfunc = ['Kalyani Nagar',
'Pimpri-Chinchwad',
'St.Thomas Mount-Pallavaram Cantonment Board',
'Kanchipuram',
'St. Thomas Mount Cantonment',
'Chitradurga',
'Gandhinagar',
'Rajpur Sonarpur',
'Chashirin',
'Khurdha Municipality Boundary',
'Municipal Corporation',
'Kalyan-Dombivli',
'New Barrackpore',]

In [53]:
adressdf['city'] = adressdf['city'].str.replace(' City| District|Greater | Rural| Suburban| Urban|East|West|North|'+'|'.join(mallfunc),'',regex=True)
adressdf['city'] = adressdf['city'].str.strip()

In [54]:
adressdf

,Search_string,lat,lon,adress,city
0,NaN,12.988360,77.580344,"{'office': 'Gandhi Bhavan', 'road': 'East Park...",Bengaluru
1,kolkata(shalimar 1),22.556790,88.315398,"{'residential': 'Merlin Waterfront', 'suburb':...",Haora
2,mumbai (all areas)(colaba 1),18.909565,72.798591,"{'road': 'Dr Homi Bhaba Road', 'neighbourhood'...",Mumbai
3,telangana(alwal 11),17.507395,78.498245,"{'neighbourhood': 'Father Balaiah Nagar', 'sub...",Hyderabad
4,delhi ncr(surajpur industrial area),28.516861,77.512748,"{'city': 'Greater Noida', 'county': 'Gautam Bu...",Noida
...,...,...,...,...,...
7622,mumbai(sector 30a vashi 1),19.072503,72.991845,"{'suburb': 'Vashi', 'city': 'Navi Mumbai', 'co...",Navi Mumbai
7623,maharashtra(nariman point),18.924799,72.818127,"{'road': 'Mumbai Coastal Road - Phase I', 'nei...",Mumbai
7624,remote (hiring office located in hyderabad),17.439563,78.374686,"{'shop': 'IKEA', 'road': 'Hi Tech City Main Ro...",Hyderabad
7625,gajuwaka,17.684982,83.186191,"{'neighbourhood': 'Visweswara Nagar', 'suburb'...",None


In [55]:
maperdic = {}
for k,v in adressdf.iterrows():
    c = v['city']
    if not isinstance(c,str) :
        c = 'nan'
    maperdic.update({v['Search_string']:v['city']})

In [56]:
adressdf[~adressdf['city'].isna()]

,Search_string,lat,lon,adress,city
0,NaN,12.988360,77.580344,"{'office': 'Gandhi Bhavan', 'road': 'East Park...",Bengaluru
1,kolkata(shalimar 1),22.556790,88.315398,"{'residential': 'Merlin Waterfront', 'suburb':...",Haora
2,mumbai (all areas)(colaba 1),18.909565,72.798591,"{'road': 'Dr Homi Bhaba Road', 'neighbourhood'...",Mumbai
3,telangana(alwal 11),17.507395,78.498245,"{'neighbourhood': 'Father Balaiah Nagar', 'sub...",Hyderabad
4,delhi ncr(surajpur industrial area),28.516861,77.512748,"{'city': 'Greater Noida', 'county': 'Gautam Bu...",Noida
...,...,...,...,...,...
7621,hyderabad(nagole 1),17.379655,78.558206,"{'road': 'Inner Ring Road', 'neighbourhood': '...",Hyderabad
7622,mumbai(sector 30a vashi 1),19.072503,72.991845,"{'suburb': 'Vashi', 'city': 'Navi Mumbai', 'co...",Navi Mumbai
7623,maharashtra(nariman point),18.924799,72.818127,"{'road': 'Mumbai Coastal Road - Phase I', 'nei...",Mumbai
7624,remote (hiring office located in hyderabad),17.439563,78.374686,"{'shop': 'IKEA', 'road': 'Hi Tech City Main Ro...",Hyderabad


In [57]:
c  = 0
for i in adressdf[adressdf['city'].isna()]['adress']:
    try:
        if i['country'] == 'India':
            print(i['district'])
        else:
            # print(i)
            None
    except:
        c+=1
    # for j in i.keys():
    #     print(j)
    # break 

In [58]:
# pd.DataFrame(locset,columns=['unique_locstions']).to_csv('location_req.csv',index=False)

In [59]:
locdic = {}
anyinlist = ['chandi']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Chandigarh'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['delh']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Delhi'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['hyd','secund']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Hyderabad'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['hmeda','ahmedabad','ahemda']#ahemdabad
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Ahmedabad'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['surat']#ahemdabad
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Surat'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['vado']#ahemdabad
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Vadodara'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['gurg']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Gurgaon'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['banga','bengalu','arnataka','angalore']
allinlist = []
notinlist = ['west']

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Bangalore'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['kochi','coch']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Kochi'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['indo']
allinlist = []
notinlist = ['indos']

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Indore'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['mumba']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Mumbai'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['pune']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Pune'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['jaipur']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Jaipur'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['chennai']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Chennai'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['coim','cbe']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Coimbatore'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['luck']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Lucknow'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['noida']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Noida'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['kolk']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Kolkata'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['thane']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
locdic['Thane'] = apl
#------------------------------------------------------------------------------------------------------


In [60]:
anyinlist = ['thane']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
# locdic['Thane'].update(apl)

{'thane'}

In [61]:
locdic.keys()

dict_keys(['Chandigarh', 'Delhi', 'Hyderabad', 'Ahmedabad', 'Surat', 'Vadodara', 'Gurgaon', 'Bangalore', 'Kochi', 'Indore', 'Mumbai', 'Pune', 'Jaipur', 'Chennai', 'Coimbatore', 'Lucknow', 'Noida', 'Kolkata', 'Thane'])

In [62]:
'|'.join(map(re.escape, locdic['Delhi']))

'delhi\\ /\\ ncr\\(\\ east\\ patel\\ nagar\\ \\)|delhi\\(\\ connaught\\ place\\ \\)|delhi\\ /\\ ncr\\(\\ ad\\ block\\ pitampura|south\\ delhi|delhi\\ /\\ ncr\\(\\ laxmi\\ nagar|delhi\\ ncr|delhi\\(\\ aerocity\\ \\)|delhi\\ /\\ ncr\\(\\ badarpur\\ \\)|new\\ delhi\\(\\ pitampura\\ \\)|new\\ delhi\\(\\ adhchini\\ \\)|delhi\\ /\\ ncr\\(\\ manesar\\ \\)|delhi\\(\\ kirti\\ nagar\\ \\)|delhi\\ /\\ ncr|delhi\\ /\\ ncr\\(\\ okhla\\ \\)|delhi\\ /\\ ncr\\(\\ aerocity\\ \\)|delhi\\(\\ malviya\\ nagar\\ \\)|delhi\\ /\\ ncr\\(\\ a\\ block\\ vikaspuri\\ \\)|delhi\\(\\ jhandewalan\\ \\)|delhi\\(\\ defence\\ colony\\ \\)|delhi\\ /\\ ncr\\(\\ naraina\\ \\)|delhi\\ /\\ ncr\\(\\ kapashera\\ \\)|delhi\\(\\ moti\\ nagar\\ \\)|new\\ delhi\\(\\ safdarjung\\ enclave\\ \\)|delhi\\ /\\ ncr\\(\\ sector\\-132\\ noida\\ \\)|delhi\\ /\\ ncr\\(\\ uttam\\ nagar|delhi\\(\\ jasola\\ \\)|delhi\\(\\ west\\ patel\\ nagar\\ \\)|delhi|delhi\\ /\\ ncr\\(\\ mahipalpur\\ extension\\ \\)|delhi\\ /\\ ncr\\(\\ udyog\\ vihar\\ \\)|

In [63]:
mappeddict = {}
for key in set(maperdic.values()):
    if (key != None) and (key != ''):
        mappeddict[key] = set()

for misp , loc in maperdic.items():
    if (loc != None) and (loc != ''):
        mappeddict[loc].add(misp)

In [64]:
def loc_maper(row, dictio):
    loclist = row.split(',')
    for loc_ind, loc in enumerate(loclist):
        for key, values in dictio.items():
            for value in values:
                if value.lower() in loc.lower():  # Case-insensitive partial match
                    loclist[loc_ind] = key
    return ','.join(loclist)

tqdm.pandas()
df['location'] = df['location'].progress_apply(loc_maper,dictio = mappeddict)
df['location'] = df['location'].progress_apply(loc_maper,dictio = locdic)

  0%|          | 120/63727 [00:01<13:13, 80.17it/s]

100%|██████████| 63727/63727 [00:28<00:00, 2199.22it/s]


In [65]:
# for loc, patterns in locdic.items():
#     pattern = '|'.join(map(re.escape, patterns))
#     try:
#         regex_pattern = re.compile(pattern)
#         df['location'] = df['location'].str.replace(regex_pattern, loc)
#     except re.error as e:
#         print(f"Error in pattern '{pattern}': {e}")

In [66]:
# for loc, patterns in mappeddict.items():
#     pattern = '|'.join(map(re.escape, patterns))
#     try:
#         regex_pattern = re.compile(pattern)
#         df['location'] = df['location'].str.replace(regex_pattern, loc)
#     except re.error as e:
#         print(f"Error in pattern '{pattern}': {e}")

In [67]:
locset2 = set()
for loc in df['location']:
    sublocs = loc.split(',')
    for subloc in sublocs:
        locset2.add(subloc.strip())
len(locset2)

521

In [68]:
# df = pd.read_excel('salary_and_company_mapping.xlsx')

In [69]:
df['mapped_job_title'] = df['job_title'].str.replace("[^a-zA-Z0-9]",' ',regex=True).str.replace('   ',' ').str.replace('  ',' ')
df['mapped_job_title'] = df['mapped_job_title'].str.lower().str.strip()

replace_lis = ['hiring b. tech freshers','hiring','b tech','freshers','hyderabad','excellent','comms','night', 'shifts','shift','day','3l ctc','ctc',
               'great', 'salaries', 'for', 'in', 'bangalore','is','tech mahindra','chennai','bca','m tech','cts','0','1','yrs','exp','urgent','cognizant',
               'graduates','joinees','immediate','cse','bulk','opening','fresher','mega','int l','sal', '4 2lpa','telugu','opportunity','going on','2 am',
               'itgc', 'big4','walk','drive','us','healthcare','ith','6','8','years','experience','with','3','6years','cust','cus','jobs','b sc','m sc',
               'mphil','biology','botany','zoology','gra','hybrid', 'remote', 'puc','grad','uk','hcl tech','looking','the leading tech firm','7lpa','tamil',
               'kannada','30 bpo','experienced','up to','btm', 'hebballocation','7','5lks','300','positions','direct','top','bpos','big','payment','company',
               'limited','pan','india','kr','malayalam','secunderabad','location','at manyata tech park','only experienced','blr','job opportunities','ca one tech',
               'infosys','payroll','ca one tech cloud','infosys','payroll','ca one tech cloud','openings','work from home','salary','13k','to','16k','amazon',
               'i','hdfc bank','the','leading telephone','brand','leading','work from office','noida','grads','share cv','upto 60','lpa','upto','sun technologies',
               'churchgate','mumbai','99acres','notice period','should be','30 45 days','30 days','45 days','one of mnc s','4 yr','mncs','wfh','wfo','iam',
               'hindi','english','joiners','less than','or less than','b e','mech','eee','required','marketing department','manyata tech','male','candidates','only',
               'job','ug','pg','pharm','lifescience','be','biotech','biomedi','work form home','call','sameera','18th aug 21st aug','5lpa','mnc','build your career',
               '4','4 2 l','uppal hafeezpeth','6lpa','7lks','8lpa','pune','gurgaon','50 old product','banaglore','hederabad','delhi','l id a02ub','secundarabad','imm',
               'joiner','blore','2pm','11pm','female','bengaluru','total rewards','rewards','iit','nit','bits','tier','walkin','interview','at','copart',
               'working from home','french','language','we are back','and','this','time','huge numbers','amazing','a','reputed organization','wroots','urgently l','urgently',
               '29k','immediately','12 months','5l','both','now','2years id a24mb','reputed','pharma industry','io pvt ltd','50 start ups','interviews','23rd aug till 25th aug',
               '27th aug 23','any','hyd','exp0 2','two way','cab 5','rotational','5l','virtual','25 aug 23','5 wrk','kpo','ahmedabad','copart','on','monday','28th','tuesday','29th',
               'eugia','shameerpet','spot','offer','ahmedabad','l2','dayshift','permanent','40k','whitefield','months','contract','role','ic','min of','home','working','mba','h r',
               'wanted','15','23','passout','mlt','operation','theatre','tech dialysis','dialysis','dialysis tech','24th','25th','aug','wanted15','23','nutrition','dietetics','dietitian',
               'nutritionist','food tech','tech guru','it open source','tech writer','journalist','all','wanted','x ray','technology mlt','zoom','19th','1st','sep',
               'kolkata','18','nanotech','operation theater','tech veterinary','sci','part ot','full','ed tech firm','anaesthesia','operation','radiology','perfusion tech','goregaon',
               '62','start your career','navi','belapur','dialysis','get','of','22k','hand','banking process','30k','50k','plus inc','20 21','one of s','an','travel','mca graduate',
               'mca','malad','name','u k based','m n c','5 days','days','work','2yrs','continental','2 continental','continental','hospital','gachibowli','generali','begumpet','subk',
               'females','27k','graduate','blended','centers','yr','req 28','req','italian experts','tech m 45','b19 sec opp dme college','e commerce','theater','scie','golden','opportunities',
               'nanotechnology','science','ii fashion','ii','hirings','hirinf','hire','hired','30th till','based','german','expert','chat process','b1 certified','australian'
               'sec 63','hr anjali','wipro','50 start ups','one s','kafka knowledge','40 k','32 k','mohali','chandigarh','co jaipur','ppc calls','swati','bfsi','chandivali','jaipur',
               'techmahindra','australian','ncr','k','month','apply','new','sat','sun','fix','off','bba','hinduja tech','kctc','dyson', 'norton','lalit','fresh','diploma','mechanical',
               'we','are','lalit','s','th','sept','en','btech','or','powai','rohtak','domestic','alisha','asap','stop', 'have', 'look','lms','oct','benefit','tamilnadu','city',
               'japan','mahipalpur',' bhosari','regional','joining','hcltech','open','position','b sec','sec','opp','dme','college','min','max','sohna','gujarat','l l','a',r'\d+k',
               '50','must 3l',r'\d+k','5 wrk','07','60','suyati technologies','kochi','iga','requirement','interested',r'\b(\w+)\b(?=.*\b\1\b)',r'^\b(?:coding|coding|coder|counselors|lead)\b$','nttdata',
               r'\b(\d+)\s*ctc\b',r'\b\d+[myMY]\b','aakash','byju','was','punjab','eia',r'\b(?:\d+\s*[lL][pP][aA]|2\s*l)\b','magnasoft',r'^\b(?:tech|process|candidate|better|associate|support|trainer|collections|medical)\b$','airoli',
               'way trasportation','portuguese','corporates gifting co','dailyrounds','domlur','manchar rajguru nagar road','datamatics andheri',r'^\b(?:internship|executive|local seo|international|intern|bpo|head)\b$',
               r'\b\d+\s\d{2}am\b',r'\bl pa\b','lucknow','hcl','thursday','engineer up','dubai','gurugram','research fellow phd scholar','retirement pension','safeai','popup calls','matrimony','com','9'
               r'\b\d+(?:st|nd|rd|th)\b','reminder','calling mandatory','strong coder','biotechnology engineers','biotechnology','genetic engineer', 'molecular','wa','em ip op',
               'perfusion','cardiac tech','physician','assistant','gradua','ophthalmic','nursing assistant','clinical care tech','b m','d pharma','Biotechnology engineer','biomedical engineer','biomed','bioscienc',
               'biotech Engineers','bvsc',r'\b\d+-\d+\b','biomedical engineer gr','2023','non technical','b m','engineers','22','gr','biomedical','bioinformatics','pharmacy','physiotherapy','nursing',
               'b m d','gradu','micro','microbiology','paramedical','bsc','msc','inorganic','organic chemistry','trauma','technology','neuro','want','vacancies','gnm','staff nurse','dgnm',
               'passoutbsc','dmlt','medical lab technician','bds','bhms','bams','bnys','bums','bsms','dentist','microbiologist','x','nurse','eligibility','verification','end rcm','biochemistry',r'\b[A-Za-z]\b',
               'physiotherapist','bpt','cell','recruiting'] 

pattern = r'\b(?:' + '|'.join(replace_lis) + r')\b'

df['mapped_job_title'] = df['mapped_job_title'].str.replace(pattern, '', regex=True)
df['mapped_job_title'] = df['mapped_job_title'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()
df['mapped_job_title'] = df['mapped_job_title'].str.replace(pattern, '', regex=True)
df['mapped_job_title'] = df['mapped_job_title'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()
# for job_title, mapped_job_title in zip(df['job_title'], df['mapped_job_title']):
#     print(f'Job Title: {job_title}, Mapped Job Title: {mapped_job_title}')

In [70]:
df['mapped_job_title_1'] = df['mapped_job_title']

In [71]:
df['mapped_job_title_1'] = df['mapped_job_title_1'].str.title() 

In [72]:
titles = df['mapped_job_title'].tolist()

In [73]:
from collections import defaultdict
title_counts = defaultdict(int)

for title in titles:
    title_counts[title] += 1
# Step 3: Sort the names based on their counts
sorted_title = sorted(title_counts.keys(), key=lambda title: title_counts[title], reverse=True)
sorted_title

titlec = []
# Print the sorted names
for title in sorted_title:
    titlec.append(f"{title}: {title_counts[title]}")


In [74]:
titledict = {}

In [75]:
skilcol = ([col for col in df.columns if 'skil' in col])

In [76]:
unique_roles = list(df['role'].unique())
# unique_roles.remove('Nan')
unique_data_list = []

for role in unique_roles:
    unique_data = {}
    unique_data['role'] = role 
    unique_data['no_of_data'] = df[df['role'] == role].shape[0]  # Count of rows for the role
    try:
        commastr =  ','.join(df[df['role'] == role][skilcol].dropna().values.tolist()[0])  # Concatenate skills into a comma-separated string
    except:
        commastr = None
    unique_data['skills'] = commastr
    unique_data_list.append(unique_data)

# Create a DataFrame from the list of unique data
unique_datadf = pd.DataFrame(unique_data_list)
unique_datadf = unique_datadf.sort_values('no_of_data',ascending=False)
unique_datadf.to_excel('unique_roles_and_skills_from_naukri_latest.xlsx',index=False)

In [77]:
df[df['role'] != 'Nan'].shape

(63727, 58)

In [78]:
df['job_title'] =df['job_title'].astype(str).replace(pd.NA,'NaN')

In [79]:
anyinlist = ['mis','management','lead','tech','executive','manager','head']
allinlist = ['mis|management|lead|manager|executive|head']
notinlist = ['admission','hemis','missi','soft','customer','voice']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]
filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Lead/Management'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['consultant']
# allinlist = ['consultant']
# notinlist = []# founder
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['consultant'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['support','tech','troubleshooting','engineer','lead','engineer','specialist','system','fluent','language']
# allinlist = []
# notinlist = ['voice','chat','customer','cust','production','analyst','devops']# founder
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['tech support'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['support','care','customer','executive','tech','troubleshooting','engineer','lead','engineer','specialist','system']
allinlist = ['customer','support|care']
notinlist = ['production','engineer','analyst','voice','chat','devops']# founder

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]
apl = set(filtered_df['job_title'].str.lower())
titledict['Customer Support'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['fluent','language','voice','call','calling','tele','chat','voice','chat','process']
allinlist = ['voice|process','voice']
notinlist = ['support']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]
apl = set(filtered_df['job_title'].str.lower())
titledict['Voice Process'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['soft','java','full','stack','back','end','developer','front','engineer','fullstack.net','dot','sap','sql']
allinlist = ['developer|engi|java|stack|soft|front|back|full|fullstack.net|dot|sap|sql','developer|engineer']
notinlist = ['data','anal']# founder

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Software Development'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['medical','pharmacist','coding','coder']
# allinlist = ['cod|medical|pharmacist|coding|coder']
# notinlist = ['equity']

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['medical coder'] = apl
# #------------------------------------------------------------------------------------------------------
# anyinlist = ['medical']
# allinlist = ['bill','med']
# notinlist = []# founder
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['medical billing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sales','inbound','tele','inside','customer','relationship','engineer','perfomance','analyst','it','tech',
             'executive','marketing']
allinlist = ['sales|executive|marketing|manager|territory']
notinlist = ['recruiter','support','care','development']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Sales'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sales','inbound','tele','inside','customer','relationship','engineer','perfomance','analyst','it','tech',
             'executive','marketing']
allinlist = ['business|development','development']
notinlist = ['recruiter','support','care','sales']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Business Development'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','anal','machine','learning','scie','data','anal','ai/ml']
allinlist = ['data|anal||scien|ai|ml']
notinlist = ['admin','video','engineer','engi']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(anyinlist) + r')', regex=True) &
                df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(allinlist) + r')', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(notinlist) + r')', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Data Scientist'] = apl

#------------------------------------------------------------------------------------------------------
anyinlist = ['data','anal','ml','machine','learning','engi','data','ai/ml']
allinlist = ['data|engi|ai|ml|engineer']
notinlist = ['admin','video','scien']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(anyinlist) + r')', regex=True) &
                df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(allinlist) + r')', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(notinlist) + r')', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Data Engineer'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['devops']
allinlist = ['devops']
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Devops'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['project','manager','soft','tele','it']
allinlist = ['project','mana',]
notinlist = ['- projects','manager project','non-it','non it']
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Project Management'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['digital','marketing','campaigns','ads','media']
allinlist = []
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Digital Marketing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['counsellor','counselor'] 
allinlist = []
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Counsellor'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['content','writer','copy']
# allinlist = ['writer']
# notinlist = []
# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['content/copy writer'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['graphic','designer']
# allinlist = ['designer']
# notinlist = []
# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['graphic designer'] =  apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['architect']
# allinlist = ['architect']
# notinlist = []
# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['architect'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['business','analyst']
allinlist = ['business','analyst']
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)

titledict['Business Analyst'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['ui','ux','designer','developer']
# allinlist = ['ui|ux']
# notinlist = ['java']

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['ui/ux designer'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['accounts']
# allinlist = ['accounts','mana']
# notinlist = ['store','ass']
# # notinlist.extend(list(titledict['it recruiter']))

# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['accounts manager'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['operations']
# allinlist = ['operations','lead|executive']
# notinlist = ['stack','front','back','soft']# founder

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['operations'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['engineer']
allinlist = ['engineer']
notinlist = []
# notinlist.extend([item for seet in titledict for item in titledict[seet]])

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Engineer'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['hr','recruiter','recru','relationship','human',]
allinlist = []
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Recruiter'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['product','management','manager']
allinlist = ['product','manager']
notinlist = ['admission','hemis','missi','soft','java','relationship','production']

# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Product Management'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['equity','dealer','finance','advisor','financial']
# allinlist = ['equity|trader|advisor|financial']
# notinlist = ['admission','hemis','missi','soft','java','relationship','production']

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['equity advisor'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['accountant','account']
# allinlist = []
# notinlist = []

# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['accountant'] = apl

In [80]:

anyinlist = ['mis']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['manager']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['lead']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate','ing','tech','net','java']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
# ,'manage','lead','executive','head'
anyinlist = ['executive','head']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate','ing','tech','net','java']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
# ,'manage','lead','executive','head'
anyinlist = ['support','care','cust','tech','troubl','engineer','specialist','system']
allinlist = ['custo']
notinlist = ['admission','hemis','missi','soft','it','mist','lyst','ern','port','list','ciate','ing','tech','net','java']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Customer Support'].update(apl)
#------------------------------------------------------------------------------------------------------
# ,'manage','lead','executive','head'
anyinlist = ['support','care','tech','troubl','engineer','specialist','system']
allinlist = ['voice']
notinlist = ['cust','missi','soft','it',]

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Voice Process'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['busine','dev']
allinlist = ['busine','dev']
notinlist = []

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Business Development'].update(apl)
#------------------------------------------------------------------------------------------------------


In [81]:
anyinlist = ['busine','dev']
allinlist = ['busine','dev']
notinlist = []

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)


In [82]:
len(titledict.keys())

16

In [83]:
for i in titledict.keys():
    print(i)

Lead/Management
Customer Support
Voice Process
Software Development
Sales
Business Development
Data Scientist
Data Engineer
Devops
Project Management
Digital Marketing
Counsellor
Business Analyst
Engineer
Recruiter
Product Management


In [84]:
df['job_title']

0                                               Pharmacist
1                                               Pharmacist
2                                        Software Engineer
3                                 Dairy Feed Sales Officer
4                                  BOAT APPRENTICE TRAINEE
                               ...                        
64215    Associate, Client Onboarding / Transitions / C...
64216              Associate Professional, Presale Support
64217              Associate Professional, Presale Support
64218              Associate Professional, Presale Support
64219              Associate Professional, Presale Support
Name: job_title, Length: 63727, dtype: object

In [85]:
anyinlist = ['accountant']
allinlist = []
notinlist = []

# notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# apl
# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# for i in apl:
#     print(i)

In [86]:
converted = pd.DataFrame.from_dict(titledict, orient='index').T
converted.to_excel('title_maps.xlsx',index = False)

In [87]:
# reverse_mapping = {title.lower(): category for category, titles in titledict.items() for title in titles}
# def map_title(value):
#     lower_value = value.lower()
#     return reverse_mapping.get(lower_value, 'unclassified')

# df['mapped_job_title'] = df['job_title'].apply(map_title)

In [88]:
df['mapped_job_title'] = 'unclassified'

In [89]:
f = []
for k,v in df[df['mapped_job_title']=='unclassified']['job_title'].value_counts().items():
    # f.append(list(k,v))
    l = []
    l.append(k)
    l.append(v)
    f.append(l)
fdf = pd.DataFrame(f,columns = ['role','count'])

In [90]:
fdf.to_excel('unmapedroles.xlsx',index = False)

In [91]:
value_counts = {}
counts = df['mapped_job_title'].value_counts()
for value, count in counts.items():
    if value in value_counts:
        value_counts[value] += count
    else:
        value_counts[value] = count
value_counts

{'unclassified': 63727}

In [92]:
# converted = pd.DataFrame.from_dict(value_counts, orient='index')
converted = pd.DataFrame.from_dict(value_counts, orient='index', columns=['Count'])
converted.index.name = "Roles"
converted.to_excel('title_value_counts.xlsx')
converted

,Count
Roles,
unclassified,63727


In [93]:
df[['job_title','mapped_job_title']].to_excel('titles and maps.xlsx',index = False)

In [94]:
df.columns

Index(['job_title', 'company_name', 'experience', 'salary', 'location',
       'description', 'posted_on', 'current_date', 'link', 'sal_range',
       'skill1', 'skill2', 'skill3', 'skill4', 'skill5', 'skill6', 'skill7',
       'role', 'industry_type', 'department', 'employment_type',
       'role_category', 'skill8', 'skill9', 'skill10', 'skill11', 'skill12',
       'skill13', 'skill14', 'skill15', 'skill16', 'skill17', 'skill18',
       'skill19', 'skill20', 'skill21', 'skill22', 'skill23', 'skill24',
       'skill25', 'skill26', 'skill27', 'skill28', 'skill29', 'skill30',
       'skill31', 'skill32', 'skill33', 'skill34', 'skill35', 'skill36',
       'skill37', 'skill38', 'skill39', 'skill40', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1'],
      dtype='object')

In [95]:
skilcol = ([col for col in df.columns if 'skil' in col])
df['combined_skills'] = df[skilcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)
df['combined_skills'] = df['combined_skills'].str.lower()
for col in df[skilcol].columns:
    df[col] = df[col].str.lower()

In [96]:
# for i in df['combined_skills']:
#     print(i)
df['combined_skills'] = df['combined_skills'].str.replace(r'^[A-Za-z0-9,.+]+$',' ',regex=True).str.lower().str.replace(r'very|good','',regex=True)
# uniqueskill = set()
# for i in df['combined_skills']:
#     tlis = i.split(',')
#     for j in tlis:
#         uniqueskill.add(j)

In [97]:
# skills = list(uniqueskill)

# json_string = json.dumps(skills, ensure_ascii=False)

# with open('uniqueskills.json', 'w', encoding='utf-8') as json_file:
#     json_file.write(json_string)

# checkpoint

In [98]:
# safeddf = df[skilcol]

In [99]:
# df[skilcol] = safeddf

In [100]:
# # removing skills that are not in the list.

# t = set()
# for i in df[skilcol]:
#     for j in df[i]:
#         try:
#             if 'skill' in j and all(word not in j for word in['python', 'it','azure','cloud','microsoft office','technical','budgeting','audit','reporting','visualization','clinical','programming','automation']):
#                 t.add(j)
#         except:
#             None
            

In [101]:
# def skilrem(value):
#     if value in t:
#         return None
#     return value

# for col in df[skilcol].columns:
#     df[col] = df[col].apply(skilrem)

In [102]:
# faulwd = set()
# t = pd.Series(list(t))
# for  i in t.str.replace(r'skills|skilled|skill|', '',regex=True).str.replace(r'[^A-Za-z0-9]', ' ',regex=True).str.strip():
#     faulwd.add(i)

# mistset = set()
# for i in faulwd:
#     i = i.split(' ')
#     for j in i:
#         mistset.add(j)

# # for i in mistset:
# #     print(i)
# mistset.discard('office')
# mistset.discard('selling')
# mistset.discard('dba')
# mistset.discard('marketing')
# mistset.discard('seo')

In [103]:
# def skilrem(value):
#     if value != None:    
#         if isinstance(value,float):
#             return None        
#         if len(value) < 2 and (value != 'c'):
#             return None 
#         if value in mistset:
#             return None
#     return value 
        
# for col in df[skilcol].columns:
#     df[col] = df[col].apply(skilrem)

In [104]:
#removal cases. rename the key to ''.

In [105]:
# errordict['hybris'] = errordict.pop('# hybris')
# errordict['sketchup'] = errordict.pop('sketchup.')
# errordict['project management'] = errordict.pop('. project management')

# payscale wise

In [106]:
skills = set()
for ind,val in df[skilcol].iterrows():
    val = val.to_list()
    skills.update(set(val))
len(skills)

30385

In [107]:
skills_edited = {}

In [108]:
anyinlist = ['statistical','analysis','stat','analyst']
allinlist = ['statis']
notinlist = ['license','meeting','purlin','designs','maint','sales','concrete','drilling','cnc','bio']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Statistical Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','modeling']
allinlist = ['model']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Modeling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['big','data']
allinlist = ['big','data']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Big Data Analytics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['deep','learning']
allinlist = ['deep','learning']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Deep Learning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['nlp','natural','language','processing']
allinlist = ['nlp']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Natural Language Processing (NLP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ai']
allinlist = ['ai','artificial']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Artificial Intelligence (AI)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','visualization']
allinlist = ['visu']
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Visualization'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tableau']
allinlist = ['tableau']
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Tableau'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sas']
allinlist = []
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri','disassembling','sass','sase','esas','sast','hsas','ssas']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAS'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['algorithm','development']
allinlist = ['algorithm','development']
notinlist = ['3d','char','2d','equipme']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Algorithm Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['computer','vision']
allinlist = ['computer','vision']
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri','disassembling','sass','sase','esas','sast','hsas','ssas']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Computer Vision'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['spark']
allinlist = ['spark']
notinlist = ['3d','char']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spark'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','min']
allinlist = ['data','min']
notinlist = ['3d','char','ministra','ming','dmin']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Mining'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['etl']
allinlist = ['etl']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ETL (Extract, transform, load)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['azure']
allinlist = ['azure']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Azure'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','warehou','dwh']
allinlist = ['dwh']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Warehouse'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','warehou']
allinlist = ['data','warehou']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Warehouse'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['google','cloud']
allinlist = ['google','cloud',]
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Cloud Platform (GCP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['gcp']
allinlist = ['gcp',]
notinlist = ['3d','char','ministra','ming','dmin','targetl','mgcp','ich']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Cloud Platform (GCP)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['power','bi']
allinlist = ['power','bi']
notinlist = ['3d','char','ministra','ming','dmin','targetl','mgcp','ich']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Power BI'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['scala']
allinlist = ['scala']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Scala'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['snowflake']
allinlist = ['snowflake']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Snowflake Cloud Data Platform'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hadoop']
allinlist = ['hadoop']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Apache Hadoop'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['devops']
allinlist = ['devops']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Development Operations (DevOps)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['automation','scripting']
allinlist = ['automation']
notinlist = ['appian', 'anywhere', 'sales', 'studio', 'azure', 'bdd', 'test', 'boardroom', 'build', 'cypress', 'engineer', 'home', 'hubspot', 'marketing', 'hvac', 'industrial', 'kotlin', 'backend', 'machine', 'marketing', 'martech', 'mis', 'office', 'perl', 'plc', 'quip', 'sap', 'security', 'smoke', 'soar', 'substation', 'ui', 'vba', 'web', 'wiser', 'workflow']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Automation Scripting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['jenkins']
allinlist = ['jenkins']
notinlist = ['sales']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Jenkins'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['terraform']
allinlist = ['terraform']
notinlist = ['sales']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Terraform'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['docker']
allinlist = ['docker']
notinlist = ['sales']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Docker'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['git']
allinlist = ['git']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['GIT'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cloud','compu']
allinlist = ['cloud','compu']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cloud Computing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['power','shell']
allinlist = ['power','shell']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows PowerShell'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sys','admin']
allinlist = ['sys','admin']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['System Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['red','hat']
allinlist = ['red','hat']
notinlist = ['sales','digi','longi','logi','gito','chatered']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Red Hat Ansible'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['recruit']
allinlist = ['recruit']
notinlist = ['sales','digi','longi','logi','gito','chatered']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Recruiting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ats']
allinlist = ['ats']
notinlist = ['cat','fat','hat','mat','lat']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Applicant Tracking System (ATS) Software'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['appli']
allinlist = ['appli','track']
notinlist = ['cat','fat','hat','mat','lat']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Applicant Tracking System (ATS) Software'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['sourcing']
allinlist = ['sourcing']
notinlist = ['garment','import','estate','resourc']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sourcing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['communi']
allinlist = ['communi']
notinlist = ['garment','import','estate','resourc','analog','unity']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Oral / Verbal Communication'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hir']
allinlist = ['hir']
notinlist = ['garment','import','estate','resourc','analog','unity','chira','ethir','fhir','third']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Hiring'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hr']
allinlist = ['hr']
notinlist = ['garment','import','estate','resourc','analog','unity','chira','ethir','fhir','third','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Human Resources (HR)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hum','res']
allinlist = ['hum','res']
notinlist = ['garment','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Human Resources (HR)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['cust','ser']
allinlist = ['cust','ser']
notinlist = ['garment','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Service'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['client']
allinlist = ['client']
notinlist = ['garment','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr','scripts','server']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Client Interaction'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sales']
allinlist = ['sales']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['peop','manag']
allinlist = ['peop','manag']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['People Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['crm']
allinlist = ['crm']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Relationship Management (CRM)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['custome','relat','manag']
allinlist = ['custome','relat','manag']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Relationship Management (CRM)'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['training']
allinlist = ['training']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Training'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['negotia']
allinlist = ['negotia']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Negotiation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['soci','media','marke']
allinlist = ['soci','media','marke']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Social Media Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['emplo','rela']
allinlist = ['emplo','rela']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Employee Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['m','offi']
allinlist = ['m','offi']
notinlist = ['sour','anal','couns','archi','hir','mid','med','manag','enga','migr','chief','front','auto','admin','ine','mily','quip','rmal','plie','ruitme',
             'tim','room']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Office'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['admin']
allinlist = ['admin']
notinlist = ['sour']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['schedu']
allinlist = ['schedu']
notinlist = ['inves','machi','produ','oject','bar']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Scheduling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['busin','dev']
allinlist = ['busin','dev']
notinlist = ['inves','machi','produ','oject','bar']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bd']
allinlist = ['bd']
notinlist = ['inves','machi','produ','oject','bar','bdd','bda','bbd','bdc','bdg','it','mbd','fbd','nbd','obd','ebd','info']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Development'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prob','sol']
allinlist = ['prob','sol']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Problem Solving'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['troublesh']
allinlist = ['troublesh']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Troubleshooting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['call','cent']
allinlist = ['call','cent']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Tech Support Call Center'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['desk','help']
allinlist = ['desk','help']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Help Desk / Desktop Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','entr']
allinlist = ['data','entr']
notinlist = ['inves','ope']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Entry'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['excel']
allinlist = ['excel']
notinlist = ['inves','ope','lence','lant','lent']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Excel'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['opera','mana']
allinlist = ['opera','mana']
notinlist = ['inves','lence','lant','lent']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Operations Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['it','suppo']
allinlist = ['it','suppo']
notinlist = ['inves','lence','lant','lent','dig','equi','onsi','recr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Information Technology (IT) Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tech','suppo']
allinlist = ['tech','suppo']
notinlist = ['inves','lence','lant','lent','dig','equi','onsi','recr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Information Technology (IT) Support'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tech','wri']
allinlist = ['tech','wri']
notinlist = ['inves',]

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Writing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bus','anal']
allinlist = ['bus','anal']
notinlist = ['inves',]

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['req','ana']
allinlist = ['req','ana']
notinlist = ['inves','mana']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Requirements Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proj','mana']
allinlist = ['proj','mana']
notinlist = ['inves',]

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Project Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proc','impr']
allinlist = ['proc','impr']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Process Improvement'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['agile']
allinlist = ['agile']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Agile Software Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sale','forc']
allinlist = ['sale','forc']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Salesforce'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prod','manag']
allinlist = ['prod','manag']
notinlist = ['ion']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['documentation']
allinlist = ['documentation']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Documentation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['visio']
allinlist = ['visio']
notinlist = ['ion','or']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Visio'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['digi','mark']
allinlist = ['digi','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Digital Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['seo']
allinlist = ['seo']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Optimization (SEO)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sear','engi','opt']
allinlist = ['sear','engi','opt']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Optimization (SEO)'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cust','sale','camp','mark']
allinlist = ['emai']
notinlist = ['comm','chat','mana']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Email Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['goog']
allinlist = ['anal']
notinlist = ['comm','chat','mana']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Analytics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web','cont']
allinlist = ['web','cont']
notinlist = ['dev']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Content Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sem']
allinlist = ['sem']
notinlist = ['emi','semb','seme','rus','tse']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Marketing (SEM)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sear','engi','mark']
allinlist = ['sear','engi','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Marketing (SEM)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['ad','word']
allinlist = ['ad','word']
notinlist = ['pad']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Ad Words'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['paid','medi']
allinlist = ['paid','medi']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Paid Media Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['stra','mark']
allinlist = ['stra','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['grap','desi']
allinlist = ['grap','desi']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Graphic Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cont','mana']
allinlist = ['cont','mana']
notinlist = ['web','act','rol']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Content Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mark','comm']
allinlist = ['mark','comm']
notinlist = ['odi','ercial']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Marketing Communications'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web','desi']
allinlist = ['web','desi']
notinlist = ['dev']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['vid','edit']
allinlist = ['vid','edit']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Video Editing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['copy','writ']
allinlist = ['copy','writ']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Copywriting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['onlin','mark']
allinlist = ['onlin','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Online Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['pay','click']
allinlist = ['pay','click']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Pay-Per-Click Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web','anal']
allinlist = ['web','anal']
notinlist = ['dev']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Analytics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['java']
allinlist = []
notinlist = ['node','react','script','js']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Java'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['java','js']
allinlist = []
notinlist = ['node','react','jso','angu','fast']
notinlist.extend( list(skills_edited['Java']))
notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Java Script'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = []
notinlist = ['my','no','pl','post','pg','ps','server']
# notinlist.extend( list(skills_edited['Java']))
# notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['python']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['Java']))
# notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Python'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['c']
allinlist = ['c']
notinlist = ['node','react','#','hash','&','/','sharp','license','meeting','purlin',
             'uce','uct','clo','co','cr','cal','nce','ce','ck','cu','cm','ch','ct','ci','ca','cl','ic','cn','cp','bc','2c','oc','ac','sc','gc','yc','vc',
             'rc','lc','ec','tc','pc','cf','cg','cs','cq','cg','cy','dc','mc','9c','qc','cd','cc','nc','wc','c1','c2','ch','cv','c4','cb','c3','cw','cx',
             'uc','fc','hc','4c','+','plus','&']
# notinlist.extend( list(skills_edited['Java']))
# notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['C Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['c']
allinlist = ['c']
notinlist = ['node','react','#','hash','&','/','sharp','license','meeting','purlin',
             'uce','uct','clo','co','cr','cal','nce','ce','ck','cu','cm','ch','ct','ci','ca','cl','ic','cn','cp','bc','2c','oc','ac','sc','gc','yc','vc',
             'rc','lc','ec','tc','pc','cf','cg','cs','cq','cg','cy','dc','mc','9c','qc','cd','cc','nc','wc','c1','c2','ch','cv','c4','cb','c3','cw','cx',
             'uc','fc','hc','4c','&']
notinlist.extend( list(skills_edited['C Programming Language']))
notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['C++ Programming Language'] = apl 
skills_edited['C++ Programming Language'].remove('c')
#------------------------------------------------------------------------------------------------------
anyinlist = ['c']
allinlist = ['c']
notinlist = ['license','meeting','purlin',
             'uce','uct','clo','co','cr','cal','nce','ce','ck','cu','cm','ch','ct','ci','ca','cl','ic','cn','cp','bc','2c','oc','ac','sc','gc','yc','vc',
             'rc','lc','ec','tc','pc','cf','cg','cs','cq','cg','cy','dc','mc','9c','qc','cd','cc','nc','wc','c1','c2','ch','cv','c4','cb','c3','cw','cx',
             'uc','fc','hc','4c','&']
notinlist.extend( list(skills_edited['C Programming Language']))
notinlist.extend( list(skills_edited['C++ Programming Language']))
notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['C# Programming Language'] = apl 
skills_edited['C# Programming Language'].remove('c')
#------------------------------------------------------------------------------------------------------
anyinlist = ['react']
allinlist = ['react']
notinlist = ['or','on']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['React.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['node']
allinlist = []
notinlist = ['fet','desi','lymph','eb']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Node.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['angul']
allinlist = []
notinlist = ['js']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Angular'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['aws']
allinlist = []
notinlist = ['law','jaw','dev']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Amazon Web Services (AWS)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['amaz','web']
allinlist = ['amaz','web']
notinlist = ['law','jaw','dev']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Amazon Web Services (AWS)'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['net','dot']
allinlist = []
notinlist = ['law','jaw','etwor','ado','bnet','trix','iler','ernet','etba','cnet','etsc','netru','inet','enet','menet','gnet','enet','vnet',
             'ranet','onet','snet','netcr','pnet','rdot','nets','netx','netw','neti','netc','netap','gh net','asp','core']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['.NET'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['linux']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Linux'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['spring']
allinlist = []
notinlist = ['boot']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spring'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['spring']
allinlist = ['boot']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spring Boot'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['gola']
allinlist = ['go']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Go (Golang) Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['kubern']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Kubernetes'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['php']
allinlist = []
notinlist = ['ephp','px','pc','ophp',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['PHP'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ruby','ror','rails']
allinlist = []
notinlist = ['rror','cror','drail','uror','oror','grail',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Ruby on Rails'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['html']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['HTML'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['machine learning','machinelearning','ml']
allinlist = []
notinlist = ['tml','aml','cont','machines','know','ancing','bsc','mlt','bml','sale','cnc','oper','messure','concre','xml','dml','hydro','drill','e-','elearning',
             'bend','measu','motiv','lamina','elec','design','garme','grin','heav','hmc','indu','inje','joom','cuttin','lathe','rawi','build','automa','assem',
             'inff','inte','mlc','mlf','mlo','qml','gml','sml','uml']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Machine Learning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data']
allinlist = ['anal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['nego']
allinlist = ['contr']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Contract Negotiation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['comp']
allinlist = ['legal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Legal Compliance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['corp']
allinlist = ['gov']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Corporate Governance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['comp']
allinlist = ['regu']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Regulatory Compliance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['litiga','case']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Litigation Case Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['contra']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Contract Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['merge','acquit']
allinlist = []
notinlist = ['eme','code','mail']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Mergers and Acquisitions'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana','cont']
allinlist = ['risk']
notinlist = ['eme','code','mail']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Risk Management / Risk Control'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['doc','revi']
allinlist = ['legal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Legal Document Review'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['intelle']
allinlist = []
notinlist = ['gent']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Intellectual Property (IP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['leade']
allinlist = ['ship']
notinlist = ['support']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Leadership'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['commercial']
allinlist = []
notinlist = ['support']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Commercial'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['real']
allinlist = ['estate']
notinlist = ['support']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Real Estate'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['litig']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Complex Litigation Case Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['lead']
allinlist = ['team','ship']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Team Leadership'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['peop']
allinlist = ['manag']
notinlist = ['connect to people']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['People Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['labor']
allinlist = ['rela']
notinlist = ['connect to people']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Labor Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['dev']
allinlist = ['prod']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['strate']
allinlist = ['plan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Planning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['strate']
allinlist = ['busi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Strategy'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prod']
allinlist = ['mark']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['rese']
allinlist = ['mark']
notinlist = ['relat']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Market Research'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana','rela']
allinlist = ['vend']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Vendor Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['des']
allinlist = ['ux']
notinlist = ['ui','lin']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['User Experience (UX) Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['anal']
allinlist = ['mark']
notinlist = ['bench','stock','equi','fin']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Market Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['engineerin']
allinlist = ['desi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Engineering Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proj']
allinlist = ['co','ordi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Project Coordinating'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['client']
allinlist = ['intera']
notinlist = ['view']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Client Interaction'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['budg',]
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Budget Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proc']
allinlist = ['improv']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Process Improvement'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['const']
allinlist = ['est']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Construction Estimating'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['assu','cont']
allinlist = ['qual']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Quality Assurance / Quality Control'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ms','micro']
allinlist = ['proj']
notinlist = ['tems']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Project'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['des','cad']
allinlist = ['aut']
notinlist = ['ted','tion','tive']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk AutoCAD'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['change']
allinlist = ['manag']
notinlist = ['ex']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Change Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['consul']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Consulting Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prog']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Program Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['organi']
allinlist = ['dev']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Organizational Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fina']
allinlist = ['anal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['serv']
allinlist = ['cust']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Service'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cent']
allinlist = ['call']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Call Center Operations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inbou']
allinlist = ['call']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Inbound Calls'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['relatio']
allinlist = ['cust']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['typing']
allinlist = []
notinlist = ['prot']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Typing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['outbo']
allinlist = ['call']
notinlist = ['prot']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Outbound Calls'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['eng']
allinlist = ['lang']
notinlist = ['engi']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['English Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['insuran']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Insurance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['medi']
allinlist = ['termi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medical Terminology'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['banking']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Banking'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['javascript']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Java Script'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inter']
allinlist = ['clien']
notinlist = ['tional']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Client Interaction'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['gen']
allinlist = ['lead']
notinlist = ['gency','ship','urgen']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Lead Generation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['b2b']
allinlist = ['sales']
notinlist = ['gency','ator','auto','hire','local','target','customer','cutting','tool','desktop','door','tolo','ptop','oto','sto']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business to Business (B2B) Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['acco']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Account Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['stra']
allinlist = ['part']
notinlist = ['admin']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Partners'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['strat']
allinlist = ['sale']
notinlist = ['admin','meet']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['insi']
allinlist = ['sale']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Inside Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tech']
allinlist = ['sale']
notinlist = ['logy','ques','ed','solut','data','math']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['sale']
notinlist = ['logy','ques','ed','solut','data','math']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sales Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['dev']
allinlist = ['prod']
notinlist = ['logy','ques','ed','solut','data','math']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['dev']
allinlist = ['web']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['js']
allinlist = ['vue']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Vue.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = ['my']
notinlist = ['amy','emy']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['MySQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['scr']
allinlist = ['typ']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Typescript'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['go']
allinlist = ['djan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Django'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['js']
allinlist = ['angular']
notinlist = ['reac','vue']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Angular.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['asp']
allinlist = ['net']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ASP.NET Framework'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cycl']
allinlist = ['ful','recru']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Full-Cycle Recruiting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['post']
allinlist = ['sql']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['PostgreSQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['db','go']
allinlist = ['mong']
notinlist = ['chef','oose']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['MongoDB'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['micro']
allinlist = ['serv']
notinlist = ['microsoft','iser']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microservices'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['nod','js']
allinlist = ['exp']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Express (Node.js)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['field']
allinlist = ['sales']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Field Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['presen']
allinlist = []
notinlist = ['presence']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Presentations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sup']
allinlist = ['desk',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Help Desk / Desktop Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag','admi']
allinlist = ['netw']
notinlist = ['menet','relation']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Network Management / Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['activ']
allinlist = ['dire']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Active Directory'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['netw']
allinlist = ['sup']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Network Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inst','upgr']
allinlist = ['soft']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Installation & Upgrade'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sup']
allinlist = ['wind']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['soft']
allinlist = ['know']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Knowledge'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['exchan']
allinlist = []
notinlist = ['case','email','admin','heat','online','eign','link','stoc','data']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Exchange'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['serv']
allinlist = ['wind']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows Server'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sale']
allinlist = ['chan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Channel Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sale']
allinlist = ['admin']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sales Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['finan']
allinlist = ['anal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['finan']
allinlist = ['mode']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Modeling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['invest']
allinlist = ['mana']
notinlist = ['reg']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Investment Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['reconciliation']
allinlist = ['reconciliation']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Reconciliation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['repo']
allinlist = ['fina']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Reporting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['investor']
allinlist = ['relat']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Investor Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['busine']
allinlist = ['intel']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Intelligence'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = ['pl']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['PL/SQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['scrip']
allinlist = ['shell']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Shell Scripting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['team']
allinlist = ['mana']
notinlist = ['time']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Team Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['merch']
allinlist = []
notinlist = ['man']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Merchandising'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['com']
allinlist = ['loan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Commercial Loans'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fin']
allinlist = ['adv']
notinlist = ['finfet']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Advisor'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['retail']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Retail'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cash']
allinlist = ['han']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cash Handling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['inven']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Inventory Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['organiz']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Organizing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fork']
allinlist = ['li']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Forklift Certified'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['span']
allinlist = ['lang']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spanish Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['maint']
allinlist = ['nance']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Maintenance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['design']
allinlist = ['reta']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cyber']
allinlist = ['sec']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cyber Security'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['risk']
allinlist = ['sec']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Risk Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inf']
allinlist = ['sec','stru']
notinlist = ['tech']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IT Security & Infrastructure'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['test','audi']
allinlist = ['sec']
notinlist = ['tech']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Testing and Auditing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['net']
allinlist = ['sec']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Network Security Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['27001']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ISO 27001'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['info','even','siem']
allinlist = ['secu']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Information and Event Management (SIEM)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['siem']
allinlist = ['siem']
notinlist = ['iemens']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Information and Event Management (SIEM)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['vulne']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Vulnerability Assessment'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['penet']
allinlist = []
notinlist = ['liqu','manua']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Penetration Testing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['risk']
allinlist = ['it']
notinlist = ['cred','commo','miti']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IT Risk'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag']
allinlist = ['it','ment']
notinlist = ['siti','dit','rit','uit','lit','fit','nit','pit','git','xit','sit','tit','cit','hit','vit','kit','inter','asset','deli','infra','oper','prod','prog','proj','erv','team','vend']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IT Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag']
allinlist = ['identity']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Identity Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['techni']
allinlist = ['servic',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Services'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['techni']
allinlist = ['analy',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['deve']
allinlist = ['softw',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['engi']
allinlist = ['syst',]
notinlist = ['air']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Systems Engineering'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bene']
allinlist = ['comp',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Benefits & Compensation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['perfo',]
notinlist = ['manager']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Performance Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hris']
allinlist = []
notinlist = ['ghris']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Human Resources Information System (HRIS)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['safe']
allinlist = ['compl']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Safety Compliance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adp']
allinlist = ['adp']
notinlist = ['tooth','staa']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ADP Payroll System'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['producti']
allinlist = ['mana']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Production Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['lean']
allinlist = ['manu']
notinlist = ['clea']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Lean Manufacturing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['machi']
allinlist = ['oper']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Machine Operation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['prod']
notinlist = ['cost']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Production Planning (SAP PP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['supp']
allinlist = ['prod','tion']
notinlist = ['phone']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Production Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cod']
allinlist = ['medi']
notinlist = ['media']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medical Coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['emr']
allinlist = []
notinlist = ['spark','aws','rush']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Electronic Medical Records (EMR)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['medi']
allinlist = ['reco']
notinlist = ['medium','mmedi','media']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medical Records'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['acc']
allinlist = ['recei']
notinlist = ['medium','mmedi','media']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Accounts Receivable'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['radiology']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Radiology'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['billing']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Billing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['claim']
allinlist = ['hand']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Claim Handling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['interven']
allinlist = ['rad']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Interventional Radiology'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['eval']
allinlist = ['mana']
notinlist = ['recr']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Evaluation and Management Auditing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['icd']
allinlist = []
notinlist = ['gicd','cicd','hicd']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['International Statistical Classifications of Diseases - 10 (ICD - 10) Coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cpt']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Current Procedural Terminology (CPT) Coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bill']
allinlist = ['coll']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Bill Collections'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['surg']
allinlist = ['code']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Surgical coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['medi']
allinlist = ['care']
notinlist = ['thcare','criti']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medicare'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['auditing']
allinlist = []
notinlist = ['tl']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Auditing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['orthopedi']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Orthopedics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['branding']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Branding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['event']
allinlist = ['manag']
notinlist = ['preven']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Event Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web']
allinlist = ['marke']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag']
allinlist = ['marketi']
notinlist = ['place']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Marketing Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['media','publ']
allinlist = ['rela']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Media / Public Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adver']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Advertising'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['event']
allinlist = ['plan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Event Planning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['soli']
allinlist = ['wor']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SolidWorks'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['elem']
allinlist = ['finit']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Finite Element Analysis (FEA)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fea']
allinlist = []
notinlist = ['feat','feas']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Finite Element Analysis (FEA)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['graph']
allinlist = ['uni']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['NX Unigraphics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['matlab']
allinlist = ['lab']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Matlab'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['verilog']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Verilog'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['catia']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['CATIA'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['revit']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk Revit MEP'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ansys']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ANSYS Simulation Software'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inven']
allinlist = []
notinlist = ['tory','ries']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk Inventor'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ios']
allinlist = []
notinlist = ['bios','tios','xios','lios','gios','dios','rios']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['iOS'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['swift']
allinlist = []
notinlist = ['plan']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Swift Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['obje']
allinlist = ['c','tive']
notinlist = ['acqu','pega','lead']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Objective-C'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['kotlin']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Kotlin Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['reac']
allinlist = ['nati']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['React Native'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['flutter']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Flutter'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mobil']
allinlist = ['develop']
notinlist = ['appi','apple','auto',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Mobile applications development'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['dart']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Dart Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ionic']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Ionic'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['androi']
allinlist = []
notinlist = ['rx']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Android Operating System Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['xamarin']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Xamarin'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sdk']
allinlist = []
notinlist = ['agile','aws','c4c','crm','face','flut','goog','image','ios','java','native','integrations','sdks']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Android Software Development Kit (SDK)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['payroll',]
allinlist = ['admi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Payroll Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bene',]
allinlist = ['admi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Benefits Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adp',]
allinlist = []
notinlist = ['head','aadp']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ADP Human Resources System'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['emp']
allinlist = ['enga']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Employee Engagement'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['m','s']
allinlist = ['word']
notinlist = ['yword','dword','wordp','sword','words','word p']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Word'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['test']
allinlist = ['metho']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Testing Methodology'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adobe']
allinlist = ['anal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe Analytics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adobe']
allinlist = ['expe']
notinlist = ['print','video']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe Experience Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['funct']
allinlist = ['test']
notinlist = ['print','video','non']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Functional testing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['test']
allinlist = ['conce']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Testing Concepts'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tele']
allinlist = ['mark']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Telemarketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ful','full']
allinlist = ['rest','web']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['RESTful Web Services'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['softw',]
allinlist = ['archi']
notinlist = ['microsoft']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Architecture'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['j2ee']
allinlist = []
notinlist = ['microsoft']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['J2EE'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['soa']
allinlist = []
notinlist = ['microsoft','soap','soar','soan','soac',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Service Oriented Architecture (SOA)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['account']
allinlist = ['sales']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Account Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['accounti']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Accounting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = ['serv']
notinlist = ['my','post']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft SQL Server'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['branch']
allinlist = ['manag']
notinlist = ['marke','operation','relation','credit']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Branch Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['flow','mana']
allinlist = ['cash','flow']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cash Flow Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prod']
allinlist = ['phot']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Photography'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['optim']
allinlist = ['social','medi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Social Media Optimization'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['blog']
allinlist = []
notinlist = ['eblog']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Blogging'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['editing']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Editing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['css']
allinlist = []
notinlist = ['icss','cssa','cssd','scss']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cascading Style Sheets (CSS)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['laravel']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Laravel Web Framework'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['jquer']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['jQuery'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['wordpress']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Wordpress'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['html5']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['HTML5'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['magento']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Magento'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['symfony']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Symfony Php Framework'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['oop']
allinlist = []
notinlist = ['coop','doop','loop','sqoop']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Object Oriented Programming (OOP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['drupal']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Drupal'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['model']
allinlist = ['view']
notinlist = ['ing']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Model-view-controller (MVC)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mvc']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Model-view-controller (MVC)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['hard']
allinlist = ['conf']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Hardware Configuration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['acti']
allinlist = ['directo']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Active Directory'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['syst']
allinlist = ['troub']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Systems Troubleshooting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['redux']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Redux'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['asynchro']
allinlist = []
notinlist = ['commu']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Asynchronous'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['ajax']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Ajax'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['xml']
allinlist = []
notinlist = ['cxml','mxml','exml']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['XML'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['boot']
allinlist = ['strap']
notinlist = ['cxml','mxml','exml']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Bootstrap'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['wire']
allinlist = ['less']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Wireless'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['ip']
allinlist = ['net','work']
notinlist = ['ship','equip']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IP Network'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['arch']
allinlist = ['net','work']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Network Architecture / Design'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['ip']
allinlist = ['tcp']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['TCP/IP'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['switching']
allinlist = ['routing']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Switching and Routing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['voip']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['VoIP'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['switch']
allinlist = ['firewall']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Switches & Firewall Installation and Administration'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['tele']
allinlist = ['com']
notinlist = ['skills']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Telecommunications'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['vm']
allinlist = ['ware','esx']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['VMware ESX'] = apl
#------------------------------------------------------------------------------------------------------

In [109]:
anyinlist = ['cisco']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cisco Networking'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['conti']
allinlist = ['inte']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Continuous Integration (CI)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['kafka']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Apache Kafka'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['hibernate']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Hibernate'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['eclipse']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Eclipse Java IDE'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['data']
allinlist = ['base','mana']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Database Management'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['ting']
allinlist = ['repor']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Reporting'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['office']
allinlist = ['admi']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Office Administration'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['phone','call','voice','chat']
allinlist = ['sup']
notinlist = ['iphone']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Phone Support'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['user','design']
allinlist = ['inter','face','user']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['User Interface Design'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['speak']
allinlist = ['publ']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Public Speaking'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['case']
allinlist = ['mana']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Case Management'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['presentation']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Presentation'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['counsel']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Counseling'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['appli']
allinlist = ['dev']
notinlist = ['apple','mobile']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Application Development'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['data']
allinlist = ['proce']
notinlist = ['sheet','dure']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Processing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sale']
allinlist = ['engi']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sales Engineering'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['enterp']
allinlist = ['sol']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Enterprise Solutions'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['prod']
allinlist = ['demon']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Demonstration'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['vb']
allinlist = ['net']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['VB.Net'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['web']
allinlist = ['api']
notinlist = ['rapi']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['WebAPI'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['dot','net']
allinlist = ['core']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['.NET Core'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['visual']
allinlist = ['basic']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Visual Basic (VB)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['vb']
allinlist = []
notinlist = ['vba','dvb','evb','vbcs','vblo']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Visual Basic (VB)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['illu']
allinlist = ['ustra','tor']
notinlist = ['vba','dvb','evb','vbcs','vblo']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe Illustrator'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['after']
allinlist = ['effe']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe After Effects'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['indesig']
allinlist = []
notinlist = ['designin']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe InDesign'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['creat']
allinlist = ['suit']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe Creative Suite'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['2d']
allinlist = ['anim']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['2D Animation'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['coral']
allinlist = ['dra']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Corel Draw'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['illus']
allinlist = ['tion']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Illustration'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['photo']
allinlist = ['shop']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe Photoshop'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['acade']
allinlist = ['advi']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Academic Advising'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['mentoring']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Mentoring'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['curri']
allinlist = []
notinlist = ['ele','int']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Curriculum Planning'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['busine']
allinlist = ['proce']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Process Engineering'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['func']
allinlist = ['anal']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Functional Analysis'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['product']
allinlist = ['supp']
notinlist = ['ion']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Support'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['chat']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Online Chat'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['purchasing']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Purchasing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['r']
allinlist = []
notinlist = ['r ','/r','r/','r&',' r','-r','.r','&r']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')
for i in range(ord('a'),ord('{')):
    notinlist.append(chr(i)+'r')
    notinlist.append('r' + chr(i))
for i in range(ord('0'),ord(':')):
    notinlist.append(chr(i)+'r')
    notinlist.append('r' + chr(i))
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['R'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['executive']
allinlist = ['supp']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Executive Support'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['data']
allinlist = ['base']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Database'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['flask']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Flask Web Framework'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['pandas']
allinlist = []
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Pandas Software Library'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['rest']
allinlist = []
notinlist = ['erest','crest','restr','irest','rrest','prest','orest','resta','resto']
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['REST'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['scm']
notinlist = []
# notinlist.extend(list(skills_edited['C Programming Language']))
# notinlist.remove('c')
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Supply Chain Management (SAP SCM)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['bi']
notinlist = ['bill','combi']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Business Intelligence (SAP BI)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['bus','intel']
notinlist = ['bill','combi']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Business Intelligence (SAP BI)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['mm']
notinlist = ['imm','amm','omm']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Materials Management (SAP MM)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['hr']
notinlist = ['thr','hrms']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Human Resources (SAP HR)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['erp']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Enterprise Resource Planning (ERP)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['abap']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP ABAP'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['sd']
notinlist = ['sdk']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Sales and Distribution (SAP SD)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['basis']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Basis'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['hana']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP High Performance Analytic Appliance (SAP HANA)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['bw']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Business Warehouse (SAP BW)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['sec']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Security'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['qm']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Quality Management (SAP QM)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['qual']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Quality Management (SAP QM)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['fico']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Financial Accounting and Controlling (SAP FICO)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['fi']
notinlist = ['fico','co','fiori','off','ca','ifi','cfi','efi']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP FI'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['succe','fac']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP SuccessFactors'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['plan','main']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Plant Maintenance (SAP PM)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['crm']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Customer Relationship Management (SAP CRM)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['mater']
allinlist = ['req','pla']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Materials Requirement Planning (MRP)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['job']
allinlist = ['sche']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Job Scheduling'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['progr']
allinlist = ['dev']
notinlist = ['web','mmer','mming','config']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Program Development'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['chil']
allinlist = ['edu']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Childhood Education'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['test']
allinlist = ['pla']
notinlist = ['mills','up']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Test Planning'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['patie']
allinlist = ['coun']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Patient Counseling'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['trau']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Emergency/Trauma'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['pre']
allinlist = ['sales']
notinlist = ['epri','pres','pret','pref','prep','real']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Pre-Sales'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['prog']
allinlist = ['api','win']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows Programming / API'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['datab']
allinlist = ['admi']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Database Administration'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['virtualiz']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Virtualization'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sys']
allinlist = ['integ']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['System Integration'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['serv']
allinlist = ['now']
notinlist = ['know']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Servicenow'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['direc']
allinlist = ['mark']
notinlist = ['indir','directing']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Direct Marketing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['cad']
allinlist = []
notinlist = ['acad','scad','ocad','o cad','d cad','auto','cadas','cade','cadw','ecad','e cad','icad','hcad','x cad','rcad','pcad','wcad','cadd']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Computer Aided Design (CAD)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['cadd']
allinlist = []
notinlist = ['acad','scad','ocad','o cad','d cad','auto','cadas','cade','cadw','ecad','e cad','icad','hcad','x cad','rcad','pcad','wcad']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Computer Aided Drafting & Design (CADD)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['proc']
allinlist = ['chem','eng']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Chemical Process Engineering'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['unit']
allinlist = ['test']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Unit Testing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['fire']
allinlist = ['base']
notinlist = ['based']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Firebase'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['xcode']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Apple Xcode IDE'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['mac']
allinlist = []
notinlist = ['mach','rmac','sup','imac','macr','maca','macy','gmac','macf','maci','macl','mace','spm']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['MacOS'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sdk','kit']
allinlist = ['ios']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['iOS SDK'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['soft']
allinlist = ['test']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Test'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['reg']
allinlist = ['test']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Regression Testing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['lab']
allinlist = ['view']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Labview'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['selenium']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Selenium Automated Test Tool'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['test']
allinlist = ['autom','tion']
notinlist = ['itest']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Test Automation'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['jmeter']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Apache JMeter'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['3d']
allinlist = ['render']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['3D Rendering'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['tech']
allinlist = ['specifi']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Specification'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['3d','studio']
allinlist = ['max']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk 3ds Max'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['archi']
allinlist = ['cad']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ArchiCAD'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sketch']
allinlist = ['up']
notinlist = ['ospace']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SketchUp'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['space']
allinlist = ['plan']
notinlist = ['ospace']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Space Planning'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['writ']
allinlist = ['proce']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Writing Procedures'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['prop']
allinlist = ['writ']
notinlist = ['life']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Proposal Writing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['proof']
allinlist = ['read']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Proofreading'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['crea']
allinlist = ['wri']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Creative Writing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['onlin']
allinlist = ['resea']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Online Research'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['alteryx']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Alteryx Designer'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['vba']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['VBA'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['elec']
allinlist = ['troub']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Electronic Troubleshooting'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['equip']
allinlist = ['maint']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Equipment Maintenance'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['prog']
allinlist = ['logic']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Programmable Logic Controllers (PLC) / Automation'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['syst']
allinlist = ['arch']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['System Architecture'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['it']
allinlist = ['consult']
notinlist = ['uit','rit','wit','sit']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IT Consulting'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['criti']
allinlist = ['care']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Critical Care'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['nursery']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Nursery'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['rehabilitation']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Rehabilitation'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['psychi']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Psychiatric'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['surgery']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Surgery'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['medicine']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medicine'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['icu']
allinlist = []
notinlist = ['ricu','ticu','micu','nicu','picu','sicu']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Intensive Care Unit (ICU)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['inte']
allinlist = ['care','unit']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Intensive Care Unit (ICU)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['phlebotomy']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Phlebotomy'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['oncology']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Oncology'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['cardiology']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cardiology'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['pediatrics']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Pediatrics'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['acute']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Acute Care'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['geriat']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Geriatrics'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['nicu']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Neonatal Intensive Care Unit (NICU)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['tele']
allinlist = ['metry']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Telemetry'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['weldi']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Welding'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['roboti']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Robotics'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['plumb']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Plumbing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['rcm']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Reliability Centred Maintenance (RCM)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['plant']
allinlist = ['main']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Plant Maintenance'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['hvac']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['HVAC System Design'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['user']
allinlist = ['rese']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['User Research'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['user']
allinlist = ['interf']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['User Interface'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['digi']
allinlist = ['desi']
notinlist = ['circu']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Digital Design'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['info']
allinlist = ['archi']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Information Architecture'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['xd']
allinlist = []
notinlist = ['xdr','axd','qxd','xdp','xdr']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Adobe XD'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['proto']
allinlist = ['type']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Prototyping'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['figma']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Figma'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['e-','ecommerce','e-commerce','e commerce']
allinlist = ['commerce']
notinlist = ['sphere','adobe','cle','orce','itecom']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['e-Commerce'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['wire']
allinlist = ['frame']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Wireframing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['inter']
allinlist = ['actio','desi']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Interaction Design'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['board']
allinlist = ['desi']
notinlist = ['story board','dashboa','test']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Board Design'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['debug']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Debug'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['fpga']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Field-Programmable Gate Array (FPGA)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['auto']
allinlist = ['cad','ele']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk AutoCAD Electrical'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['altium']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Altium Designer'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['japa']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Japanese Language'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['clini']
allinlist = ['resea']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Clinical Research'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['reta']
allinlist = ['phar']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Retail Pharmacy'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['care']
notinlist = ['career']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Managed Care'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['pati']
allinlist = ['educ']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Patient Education'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['phar']
allinlist = ['produ']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Pharmaceutical Production'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['resea']
allinlist = ['anal']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Research Analysis'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['applicatio']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Applications'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['portfo']
allinlist = ['mana']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Portfolio Management'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['facili']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Facilitator'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['valuat']
allinlist = []
notinlist = ['eval']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Valuation'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = ['lite']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SQLite'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['driv']
allinlist = ['test','dev']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Test Driven Development (TDD)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['tdd']
allinlist = []
notinlist = ['atd','bdd']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Test Driven Development (TDD)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['os','oper']
allinlist = ['windows']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows Operating System General Use'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['rhel']
allinlist = []
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Red Hat Enterprise Linux (RHEL)'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['serv']
allinlist = ['window','12']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows Server 2012 R2'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['oracl']
allinlist = ['sql']
notinlist = ['pl']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Oracle SQL'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['oracl']
allinlist = ['appl','exp']
notinlist = []
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Oracle Application Express'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['erp']
allinlist = []
notinlist = ['inter','unterp','werp','tterp','serp','verp','terp']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Enterprise Resource Planning (ERP)'] = apl
#------------------------------------------------------------------------------------------------------


In [110]:
anyinlist = ['erp']
allinlist = []
notinlist = ['inter','unterp','werp','tterp','serp','verp','terp']
    
apl = set()
for skill in skills:
    if isinstance(skill,str):
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
# skills_edited['Enterprise Resource Planning (ERP)'] = apl

{'baan erp',
 'cloud erp',
 'erp',
 'erp administrator',
 'erp apps dba',
 'erp cloud',
 'erp consultant',
 'erp consulting',
 'erp dba',
 'erp environment',
 'erp executive',
 'erp functional',
 'erp hiring',
 'erp implementation',
 'erp knowledge',
 'erp ln',
 'erp sales',
 'erp scm modules',
 'erp security',
 'erp solution',
 'erp solution sales',
 'erp system',
 'erp systems',
 'erp testing',
 'erp tool',
 'erp upload',
 'erps',
 'ifs erp',
 'integration with erp & crm is an added advantage',
 'microsoft dynamics erp',
 'msd erp',
 'netsuite erp',
 'oracle cloud erp',
 'oracle erp',
 'oracle erp cloud',
 'oracle erp domain',
 'sap erp',
 'sap erp implementation',
 'sap erp logistics',
 'sap erp sd outbound processes',
 'sap erp solutions',
 'sap hcm erp6',
 'sd erp',
 'shiperp',
 'sun6 erp',
 'tally erp'}

In [111]:
l = '''
Enterprise Resource Planning (ERP)
SQL
Oracle SQL
Database
Microsoft SQL Server
C# Programming Language
PL/SQL
Python
Data Modeling
Shell Scripting
Java Script
MySQL
IT Consulting
IT Management
Oracle Application Express
System Architecture
.NET
Software Architecture
Data Analysis
Business Analysis
'''
l = l.strip().split('\n')
l = set(l)

faul = {'Autodesk Revit Architecture','Tableau Software','Apache Spark','Price Quotes','Strategy','Influencer Marketing','Computer Hardware Technician',
        'Loan Underwriting','Clinical Assessment','Manufacturing Process Engineering','Well Production Engineering',
        'Electronic Equipment Installation','Mechanical Tool Repair','Pro/Engineer','System Repair','Enterprise Application Integration',
        'Labor & Delivery, Birthing','Recovery / Post-Anesthesia Care Unit (PACU)','Operating Room / OR','Long Term Care','Emergency Room (ER)',
        'Facility Maintenance & Coordination','Clinical Education','Hospital Education','Mail Order Pharmacy','Community Health','Pharmacotherapy',
        'Training Program Development','Bloomberg Terminal','Equity Management'}
c = 0
for i in l:
    if all(word.lower() != i.lower() for word in skills_edited) and all(word.lower() != i.lower() for word in faul):
        c += 1
        print(c,'-', i)
print(l)

{'Data Modeling', 'Python', 'PL/SQL', 'SQL', 'Java Script', 'IT Management', 'Oracle Application Express', 'IT Consulting', 'Data Analysis', 'Microsoft SQL Server', 'Enterprise Resource Planning (ERP)', 'MySQL', 'C# Programming Language', 'Software Architecture', 'Business Analysis', '.NET', 'Shell Scripting', 'Oracle SQL', 'System Architecture', 'Database'}


In [112]:
c = 0
for i in skills_edited:
    if 'windo' in i.lower():
        c += 1
        print(c,'-', i)

1 - Windows PowerShell
2 - Windows Support
3 - Windows Server
4 - Windows Programming / API
5 - Windows Operating System General Use
6 - Windows Server 2012 R2


In [113]:
len(skills_edited)

489

In [114]:
reverse_mapping = {title.lower(): category for category, titles in skills_edited.items() for title in titles}
def map_title(value):
    if value is None:
        return None
    if isinstance(value, float):
        return None 
    else:
        if (len(value)>35) or value == '':
            return None 
        if (len(value)<2) and (value != 'c'):
            return None 
        lower_value = value.lower()
        return reverse_mapping.get(lower_value)

for i in skilcol:
    df[i] = df[i].apply(map_title)
    df[i] = df[i].apply(lambda x: None if len(str(x))<2 else x)
    df[i] = df[i].apply(lambda x: None if str(x).isdigit() else x)

In [115]:
df['combined_skills'] = df[skilcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)

In [116]:
skills_filter = ['Business Development','Client Interaction','New Business Development','Lead Generation','Business to Business (B2B) Sales','Oral / Verbal Communication','Market Research','Sales','Project Management','Customer Relationship Management (CRM)','Account Management','Strategic Partners','Strategic Sales','Negotiation','Inside Sales','Technical Sales','Marketing Communications','Strategic Marketing','Sales Management','Product Development']

any_title_filter = ['busine','sale','inside','outside']
all_title_filter = ['exec']
not_title_filter = ['java']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Business Development'
#------------------------------------------------------------------------------------------------------
skills_filter = ['JavaScript','React.js','Node.js','SQL','Java','Angular','.NET','Python','PHP','C# Programming Language','Web Development','Amazon Web Services (AWS)','Vue.js','Typescript','MySQL','Spring','Django','Angular.js','Ruby on Rails','ASP.NET Framework']

any_title_filter = ['stac','dev']
all_title_filter = ['full']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Full Stack Developement'
#------------------------------------------------------------------------------------------------------
skills_filter = ['SQL','JavaScript','C# Programming Language','Java','.NET','Python','React.js','Angular','C++ Programming Language','Node.js','PHP','Amazon Web Services (AWS)','Typescript','HTML','Spring','MySQL','Linux','C Programming Language','Ruby on Rails','Go (Golang) Programming Language']

any_title_filter = ['dev']
all_title_filter = ['soft']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Software Development'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Sales','Microsoft Office','Customer Service','Outbound Calls']

any_title_filter = ['tele']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Tele Sales'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Recruiting','Full-Cycle Recruiting','Applicant Tracking System (ATS) Software','Sourcing','Oral / Verbal Communication','Hiring','Human Resources (HR)','Customer Service','Client Interaction','Sales','People Management','Customer Relationship Management (CRM)','Training','Negotiation','Social Media Marketing','Employee Relations','Microsoft Office','Scheduling','Administration','Business Development']

any_title_filter = ['recru','tale']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Recruitment & Talent Acquisition'
#------------------------------------------------------------------------------------------------------
skills_filter = ['JavaScript','Node.js','Python','SQL','MySQL','Java','PHP','Amazon Web Services (AWS)','PostgreSQL','.NET','C# Programming Language','MongoDB','Microservices','Go (Golang) Programming Language','Django','Ruby on Rails','Typescript','Linux','Express (Node.js)','Spring']

any_title_filter = ['dev','end']
all_title_filter = ['back']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Back End Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Field Sales','Business to Business (B2B) Sales','Sales','Account Management','Marketing Communications','Customer Relationship Management (CRM)','Strategic Sales','Technical Sales','Customer Relations','Sales Management','Business Development','Customer Service','Project Management','Training','Negotiation','Presentations','New Business Development','Oral / Verbal Communication','Microsoft Excel','Operations Management']

any_title_filter = ['exec','sales']
all_title_filter = ['field']
not_title_filter = ['officer']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Field Sales Executive'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Information Technology (IT) Support','Troubleshooting','Help Desk / Desktop Support (Tier 2)','System Administration','Help Desk / Desktop Support','Computer Hardware Technician','Customer Service','Network Management / Administration','Microsoft Active Directory','Help Desk / Desktop Support (Tier 3)','Network Support','Systems Troubleshooting','Software Installation & Upgrade','Project Management','Microsoft Office','PC Support','Software Knowledge','Windows Support','Microsoft Exchange','Windows Server 2012 R2']

any_title_filter = ['info','tech','sup']
all_title_filter = ['it']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'IT Support'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Sales Management','New Business Development','Strategic Sales','Customer Relationship Management (CRM)','Business Development','Account Management','Field Sales','Sales','Business to Business (B2B) Sales','Customer Service','Technical Sales','Negotiation','Customer Relations','Contract Negotiation','Inside Sales','Salesforce','Oral / Verbal Communication','Marketing Communications','Channel Sales','Project Management']

any_title_filter = ['sale']
all_title_filter = ['dire']
not_title_filter = ['officer']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Direct Sales Executive'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Customer Service','Sales Administration','Customer Relationship Management (CRM)','Microsoft Excel','Microsoft Office','Salesforce','Customer Relations','Data Entry','Database Management & Reporting','Process Improvement','Account Management','Data Analysis','Oral / Verbal Communication','Inside Sales','Project Management','Proposal Writing','Pricing','Sales','Training','Presentations']

any_title_filter = ['sale']
all_title_filter = ['supp','oper']
not_title_filter = ['officer']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Sales Support & Operations'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Financial Analysis','Microsoft Excel','Financial Modeling','Operations Management','Risk Management / Risk Control','Mergers and Acquisitions','Banking','Project Management','Investment Management','Data Analysis','Reconciliation','Financial Reporting','Customer Relationship Management (CRM)','Customer Service','Microsoft Office','Investor Relations','Python','Business Intelligence','PL/SQL','Shell Scripting']

any_title_filter = ['bank','trad','inves']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'BFSI, Investments & Trading'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Field Sales','Sales','Business to Business (B2B) Sales','Account Management','Marketing Communications','Customer Relationship Management (CRM)','Strategic Sales','Technical Sales','Customer Relations','Sales Management','Business Development','Customer Service','Project Management','Training','Negotiation','Presentations','New Business Development','Oral / Verbal Communication','Microsoft Excel','Operations Management']

any_title_filter = ['area']
all_title_filter = []
not_title_filter = ['officer']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Area Sales Manager (B2B)'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Customer Relationship Management (CRM)','Sales','Customer Relations','Client Interaction','Account Management','Financial Analysis','Oral / Verbal Communication','Business Development','Banking','Project Management','Commercial Loans','Financial Advisor','Customer Service','Sales Management','Marketing Communications','Negotiation','Salesforce','Loan Underwriting','Program Management','Human Resources (HR)']

any_title_filter = ['rela']
all_title_filter = ['mana']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Relationship Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Customer Service','Retail','Cash Handling','Merchandising','Customer Relations','Client Interaction','Oral / Verbal Communication','Sales','Inventory Management','Organizing','Training','Leadership','Forklift Certified','Spanish Language','Maintenance','Sales Management','Customer Relationship Management (CRM)','Design','Inside Sales','Microsoft Office']

any_title_filter = ['reta']
all_title_filter = ['sal']
not_title_filter = ['officer']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Retail Sales'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Cyber Security','Security Risk Management','Security Policies and Procedures','IT Security & Infrastructure','Security Testing and Auditing','Risk Management / Risk Control','Network Security Management','ISO 27001','Computer Security','Team Management','Security Information and Event Management (SIEM)','Vulnerability Assessment','ISO/IEC 27001','Project Management','Regulatory Compliance','Amazon Web Services (AWS)','Penetration Testing','IT Risk','IT Management','Identity Management']

any_title_filter = ['secu']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'IT & Information Security'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Technical Services','Technical Analysis','Systems Troubleshooting','Project Management','Information Technology (IT) Support','Software Development','Engineering Design','Data Analysis','Electronic Troubleshooting','Microsoft Office','Training','Microsoft Azure','Python','Java','Product Development','.NET','Development Operations (DevOps)','Systems Engineering','C Programming Language','Technical Writing']

any_title_filter = ['tech']
all_title_filter = ['lead']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Tech Lead'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Human Resources (HR)','Employee Relations','Benefits & Compensation','Performance Management','Recruiting','Payroll Administration','Full-Cycle Recruiting','Generalist Duties','People Management','Human Resources Information System (HRIS)','Strategic Planning','Organizational Development','Legal Compliance','Benefits Administration','Training','Labor Relations','Hiring','Safety Compliance','ADP Payroll System','Microsoft Excel']

any_title_filter = ['hr','human']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Human Resources'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Production Management','People Management','Problem Solving','Leadership','Team Leadership','Lean Manufacturing','Quality Assurance / Quality Control','Microsoft Office','Training','Process Improvement','Machine Operation','Scheduling','Safety Compliance','Operations Management','Microsoft Excel','SAP Production Planning (SAP PP)','Oral / Verbal Communication','Project Management','Maintenance','Production Support']

any_title_filter = ['prod']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Production & Manufacturing'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Medical Coding','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Current Procedural Terminology (CPT) Coding','Electronic Medical Records (EMR)','Billing','Surgical coding','Insurance','Medical Terminology','Medical Records','Epic Health Information Management System','Emergency Room (ER)','Auditing','Interventional Radiology','Quality Assurance / Quality Control','Orthopedics','Evaluation and Management Auditing','Microsoft Office','Emergency/Trauma','Radiology','Medicaid & Medicare Billing','Billing','Medicaid & Medicare Billing','Insurance','Accounts Receivable','Data Entry','Medical Coding','Bill Collections','Claim Handling','Electronic Medical Records (EMR)','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Medicaid Billing','Medical Terminology','Epic Hospital Billing','Office Administration','Medicare','Medical Credentialing','Current Procedural Terminology (CPT) Coding','Microsoft Excel','Team Leadership','Microsoft Office'}

any_title_filter = ['bill']
all_title_filter = ['medi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Medical Biller'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Medical Coding','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Current Procedural Terminology (CPT) Coding','Electronic Medical Records (EMR)','Billing','Surgical coding','Insurance','Medical Terminology','Medical Records','Epic Health Information Management System','Emergency Room (ER)','Auditing','Interventional Radiology','Quality Assurance / Quality Control','Orthopedics','Evaluation and Management Auditing','Microsoft Office','Emergency/Trauma','Radiology','Medicaid & Medicare Billing','Billing','Medicaid & Medicare Billing','Insurance','Accounts Receivable','Data Entry','Medical Coding','Bill Collections','Claim Handling','Electronic Medical Records (EMR)','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Medicaid Billing','Medical Terminology','Epic Hospital Billing','Office Administration','Medicare','Medical Credentialing','Current Procedural Terminology (CPT) Coding','Microsoft Excel','Team Leadership','Microsoft Office'}

any_title_filter = ['code']
all_title_filter = ['medi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Medical Coder'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Marketing Management','Strategic Marketing','Marketing Communications','Social Media Marketing','Project Management','Digital Marketing','Email Marketing','Online Marketing','Graphic Design','Content Management','Branding','Advertising','Event Management','Data Analysis','Search Engine Optimization (SEO)','Event Planning','Market Research','Media / Public Relations','Marketing Administration','Web Marketing'}

any_title_filter = ['market','busi']
all_title_filter = ['manag']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Marketing Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Engineering Design','Computer Aided Design (CAD)','SolidWorks','Product Development','Autodesk AutoCAD','Microsoft Office','Microsoft Excel','Project Management','CATIA','Design','Finite Element Analysis (FEA)','Autodesk Inventor','Pro/Engineer','NX Unigraphics','Computer Aided Drafting & Design (CADD)','Autodesk AutoCAD Civil 3d','ANSYS Simulation Software','Matlab','Autodesk Revit MEP','verilog'}

any_title_filter = ['desi']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Design Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Sales Management','Team Management','Customer Relationship Management (CRM)','Team Leadership','Operations Management','Field Sales','Strategic Sales','Business Development','Sales','Technical Sales','Insurance','Customer Relations','Business to Business (B2B) Sales','Strategic Project Management','Customer Service','Budget Management','People Management','Performance Management','Project Management','Leadership'}

any_title_filter = ['branc','b2c']
all_title_filter = []
not_title_filter = ['officer']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Branch Sales Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Mobile applications development','iOS','Android Software Development Kit (SDK)','flutter','React Native','JavaScript','Java','Xamarin','Swift Programming Language','Kotlin Programming Language','React.js','Dart Programming Language','C# Programming Language','PL/SQL','Ionic','Angular','Typescript','.NET','Objective-C','Android Operating System Development'}

any_title_filter = ['app','mobi']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Mobile / App Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Human Resources (HR)','Employee Relations','Recruiting','Human Resources Information System (HRIS)','Benefits Administration','Benefits & Compensation','Full-Cycle Recruiting','Payroll Administration','Performance Management','Leave of Absence Administration','Training','Legal Compliance','Hiring','Project Management','ADP Payroll System','Microsoft Office','ADP Human Resources System','Microsoft Excel','Microsoft Word','Employee Engagement'}

any_title_filter = ['hr']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'HR Generalist'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Adobe Experience Management','Adobe Analytics','Testing Concepts','Testing Methodology','Functional Testing'}

any_title_filter = ['']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'IT Infrastructure Services'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Lead Generation','Customer Relationship Management (CRM)','Salesforce','Strategic Marketing','Team Leadership','Microsoft Excel','Business Development','People Management','Online Marketing','Social Media Marketing','Sales','Marketing Communications','Marketing Management','Telemarketing','Operations Management','Google Analytics','Search Engine Optimization (SEO)','Data Analysis','Budget Management','Data Entry'}

any_title_filter = ['lead']
all_title_filter = ['gen']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Lead Generation'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Software Architecture','Cloud Computing','RESTful Web Services','J2EE','.NET','Microsoft Azure','Java','Amazon Web Services (AWS)','Salesforce','SQL','JavaScript','Design','Microservices','Autodesk AutoCAD','Python','Service Oriented Architecture (SOA)','Autodesk Revit Architecture','Linux','React.js','C# Programming Language'}

any_title_filter = ['arch']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Technical Architect'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Account Management','Client Interaction','Customer Relationship Management (CRM)','Project Management','Customer Service','Account Sales','Customer Relations','Data Analysis','Business Development','Accounting','Sales','Insurance','Microsoft Excel','Digital Marketing','Marketing Communications','Operations Management','Microsoft Office','Technical Sales','Contract Negotiation','Strategic Sales'}

any_title_filter = ['accou']
all_title_filter = ['mana']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Account Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = {'SQL','Python','ETL (Extract, transform, load)','Amazon Web Services (AWS)','Spark','Data Analysis','Microsoft Azure','Big Data Analytics','Data Warehouse','Google Cloud Platform (GCP)','Microsoft Power BI','Data Modeling','Machine Learning','Microsoft SQL Server','Scala','Snowflake Cloud Data Platform','Software Development','Java','Tableau','Apache Hadoop'}

any_title_filter = ['data']
all_title_filter = ['engine']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Data Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Sales','Customer Service','Field Sales','Sales Management','Business to Business (B2B) Sales','Business Development','Customer Relationship Management (CRM)','Banking','Microsoft Office','Branch Management','Channel Sales','Marketing Communications','Retail','Cash Flow Management','Price Quotes'}

any_title_filter = ['officer']
all_title_filter = ['sales']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Sales Officer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Social Media Marketing','Content Management','Social Media Optimization','Copywriting','Graphic Design','Marketing Communications','Video Editing','Digital Marketing','Blogging','Strategy','Strategic Planning','Media / Public Relations','Product Photography','Influencer Marketing','Paid Media Marketing','Project Management','Editing','Web Content Management','Strategic Marketing','Data Analysis'}

any_title_filter = ['media',]
all_title_filter = ['soci']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Social Media Marketing'
#------------------------------------------------------------------------------------------------------
skills_filter = {'PHP','MySQL','Laravel Web Framework','JavaScript','Model-view-controller (MVC)','jQuery','HTML5','Object Oriented Programming (OOP)','Magento','Wordpress','Vue.js','HTML','SQL','Symfony Php Framework','PostgreSQL','Drupal','Docker','Linux','Cascading Style Sheets (CSS)','Web Development'}

any_title_filter = ['php']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Php Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Help Desk / Desktop Support','Software Installation & Upgrade','Information Technology (IT) Support','Windows Support','Help Desk / Desktop Support','Microsoft Active Directory','Computer Hardware Technician','Troubleshooting','System Administration','Network Support','Active Directory','Hardware Configuration','Systems Troubleshooting','Systems Engineering','Microsoft Office','IT Security & Infrastructure','SQL','Sales'}

any_title_filter = ['desk']
all_title_filter = ['supp']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Desktop Support Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'React.js','JavaScript','HTML5','Node.js','Typescript','Cascading Style Sheets (CSS)','Web Development','Redux','React Native','HTML','Python','Java','JScript','Asynchronous','XML ','Ajax','User Experience (UX) Design','MySQL','Web Design','.NET','MongoDB','Bootstrap'}

any_title_filter = ['devel']
all_title_filter = ['reac']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'React JS Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Switching and Routing','Cisco Networking','Network Management / Administration','Network Support','Switches & Firewall Installation and Administration','Network Architecture / Design','Telecommunications','Wireless','IT Security & Infrastructure','VoIP','Microsoft Active Directory','VMware ESX','Network Security Management','TCP/IP','Linux','Amazon Web Services (AWS)','IP Network','Python','Microsoft Office','Cloud Computing'}

any_title_filter = ['network']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Network Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'JavaScript','SQL','Python','Java','C# Programming Language','React.js','C++ Programming Language','Amazon Web Services (AWS)','.NET','Node.js','Angular','Linux','Typescript','C Programming Language','Spring','Go (Golang) Programming Language','Kubernetes','PHP','Ruby on Rails','HTML'}

any_title_filter = ['soft']
all_title_filter = ['engi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Software Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Development Operations (DevOps)','Amazon Web Services (AWS)','Kubernetes','Linux','Automation Scripting','Jenkins','Python','Microsoft Azure','Terraform','Docker','Continuous Integration (CI)','Cloud Computing','Windows PowerShell','Google Cloud Platform (GCP)','Software Development','System Administration','Red Hat Ansible','Git','SQL','Shell Scripting'}

any_title_filter = ['devo']
all_title_filter = ['engi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Devops Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Java','Spring','RESTful Web Services','SQL','J2EE','Hibernate','Microservices','JavaScript','Angular','Spring Boot','Amazon Web Services (AWS)','Web Development','Agile Software Development','MySQL','Python','React.js','Apache Kafka','HTML','Eclipse Java IDE','Microsoft Azure'}

any_title_filter = ['java']
all_title_filter = ['deve']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Java Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Sales Administration','Customer Service','Customer Relationship Management (CRM)','Microsoft Excel','Microsoft Office','Customer Relations','Data Entry','Sales','Sales Management','Database Management','Reporting','Project Management','Organizing','Data Analysis','Account Management','Salesforce','Event Planning','Office Administration','Marketing Communications','Phone Support','Organizational Development'}

any_title_filter = ['coordi']
all_title_filter = ['sales']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Sales Coordinator'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Java Script','React.js','Cascading Style Sheets (CSS)','HTML5','Web Development','Typescript','Angular','Vue.js','Node.js','User Experience (UX) Design','Wordpress','PHP','Angular.js','React Native','User Interface Design','Git','jQuery','HTML','Bootstrap','Redux'}

any_title_filter = ['front']
all_title_filter = ['deve']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Front End Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Oral / Verbal Communication','Recruiting','Customer Service','Counseling','Public Speaking','Sales','Microsoft Office','Data Processing','Office Administration','Presentation','Project Management','Leadership','Event Planning','Data Entry','Salesforce','Marketing Communications','Case Management','Customer Relationship Management (CRM)','Application Development','Documentation'}

any_title_filter = ['admis']
all_title_filter = ['coun']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Admission Counsellor'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Sales Engineering','Technical Sales','Customer Relationship Management (CRM)','Presentations','Sales','Project Management','Engineering Design','Business Development','Customer Relations','Product Demonstration','Account Management','Microsoft Office','Enterprise Solutions','Cyber Security','Marketing Communications','Product Management','Strategic Sales','Autodesk AutoCAD','Microsoft Excel','Salesforce'}

any_title_filter = ['sale']
all_title_filter = ['engi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Sales Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'C# Programming Language','.NET','Microsoft SQL Server','ASP.NET Framework','SQL','.NET Core','Model-view-controller (MVC)','Java Script','Microsoft Azure','Angular','React.js','Web Development','WebAPI','Microservices','Amazon Web Services (AWS)','Xamarin','Visual Basic (VB)','RESTful Web Services','VB.Net','Angular.js'}

any_title_filter = ['net']
all_title_filter = ['deve']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Dot Net Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Graphic Design','Adobe Illustrator','Adobe Photoshop','Adobe Creative Suite','Branding','Adobe InDesign','Web Design','Design','Social Media Marketing','Video Editing','Illustration','Marketing Communications','Project Management','Adobe After Effects','User Experience (UX) Design','Product Photography','Corel Draw','2D Animation','Microsoft Office','HTML'}

any_title_filter = ['desi']
all_title_filter = ['grap']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Graphic Designer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Counseling','Oral / Verbal Communication','Mentoring','Academic Advising','Microsoft Office','Curriculum Planning','Customer Relationship Management (CRM)','Sales','Public Speaking','Program Management','Leadership','Data Processing','Documentation','Clinical Assessment','English Language'}

any_title_filter = ['acad']
all_title_filter = ['coun']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Academic Counsellor'
#------------------------------------------------------------------------------------------------------
skills_filter = skills_filter = {'Counseling','Oral / Verbal Communication','Mentoring','Academic Advising','Microsoft Office','Curriculum Planning','Customer Relationship Management (CRM)','Sales','Public Speaking','Program Management','Leadership','Data Processing','Documentation','Clinical Assessment','English Language'}

any_title_filter = ['busi']
all_title_filter = ['anal']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Business Analyst'
#------------------------------------------------------------------------------------------------------
skills_filter = skills_filter = {'Customer Service','Problem Solving','Phone Support','Troubleshooting','Customer Relationship Management (CRM)','Microsoft Office','Tech Support Call Center','Product Support','Oral / Verbal Communication','Online Chat','English Language','Help Desk / Desktop Support (Tier 1)','Customer Relations','Data Entry','Microsoft Excel','Operations Management','Banking','Information Technology (IT) Support','Technical Writing','Spanish Language'}

any_title_filter = ['cust']
all_title_filter = ['supp']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Customer Support'
#------------------------------------------------------------------------------------------------------
skills_filter = skills_filter = {'Inside Sales','Customer Service','Customer Relations','Account Management','Customer Relationship Management (CRM)','Technical Sales','Account Sales','Sales','Data Entry','Oral / Verbal Communication','Business Development','Purchasing','Strategic Sales','Negotiation','Software Knowledge','Salesforce','Microsoft Office','Sales Management','Microsoft Excel','Spanish Language'}

any_title_filter = ['inside']
all_title_filter = ['sale']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Inside Sales'
#------------------------------------------------------------------------------------------------------
skills_filter = skills_filter = {'Python','Machine Learning','Data Analysis','Statistical Analysis','SQL','R','Data Modeling','Big Data Analytics','Deep Learning','Natural Language Processing (NLP)','Artificial Intelligence (AI)','Data Visualization','Software Development','Computer Vision','Tableau Software','SAS','Algorithm Development','Apache Spark','Amazon Web Services (AWS)','Data Mining'}

any_title_filter = ['data']
all_title_filter = ['scien']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Data Scientist'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Customer Service','Oral / Verbal Communication','Inbound Calls','Problem Solving','Customer Relations','Customer Relationship Management (CRM)','Microsoft Excel','English Language','Project Management','Team Leadership','Executive Support','Call Center Operations','Microsoft Office','Data Entry','Training','Phone Support','Accounts Receivable','Office Administration','Strategic Planning','Account Management'}

any_title_filter = ['cust']
all_title_filter = ['care']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Customer Care'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Python','Django','SQL','Software Development','Amazon Web Services (AWS)','Data Analysis','Flask Web Framework','Java Script','REST','Docker','Kubernetes','MySQL','PostgreSQL','RESTful Web Services','Pandas Software Library','Linux','HTML','Database','Machine Learning','Web Development'}

any_title_filter = ['python']
all_title_filter = ['dev']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Python Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Recruiting','Full-Cycle Recruiting','Applicant Tracking System (ATS) Software','Sourcing','Oral / Verbal Communication','Hiring','Human Resources (HR)','Customer Service','Client Interaction','Sales','People Management','Customer Relationship Management (CRM)','Training','Negotiation','Social Media Marketing','Scheduling','Employee Relations','Microsoft Office','Administration','Business Development'}

any_title_filter = ['recru']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Recruiter'
#------------------------------------------------------------------------------------------------------
skills_filter = {'SAP Materials Management (SAP MM)','SAP Sales and Distribution (SAP SD)','SAP Financial Accounting and Controlling (SAP FICO)','SAP Enterprise Resource Planning (ERP)','SAP ABAP','SAP High Performance Analytic Appliance (SAP HANA)','SAP Production Planning (SAP PP)','SAP Human Resources (SAP HR)','Project Management','Business Analysis','SAP Plant Maintenance (SAP PM)','SAP Basis','SAP Business Warehouse (SAP BW)','SAP Supply Chain Management (SAP SCM)','SAP Business Intelligence (SAP BI)','SAP Security','SAP Quality Management (SAP QM)','SAP Customer Relationship Management (SAP CRM)','SAP FI','SAP SuccessFactors'}

any_title_filter = ['sap']
all_title_filter = ['cons']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Sap Consultant'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Production Management','People Management','Operations Management','Leadership','Lean Manufacturing','Quality Assurance / Quality Control','Job Scheduling','Process Improvement','Strategic Planning','Project Management','Product Development','Manufacturing Process Engineering','Materials Requirement Planning (MRP)','Scheduling','Machine Operation','Maintenance','Production Support','SAP Production Planning (SAP PP)','Organizational Development','Microsoft Excel'}

any_title_filter = ['produc']
all_title_filter = ['mana','tion']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Production Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Strategic Sales', 'Sales Management', 'Customer Service', 'Strategic Marketing', 'Field Sales', 'Customer Relationship Management (CRM)', 'Sales', 'Presentations', 'Business Development', 'Oral / Verbal Communication', 'Salesforce', 'Product Demonstration', 'Business to Business (B2B) Sales', 'Technical Sales', 'Customer Relations', 'Pre-Sales', 'Account Management', 'Marketing Communications', 'Inside Sales', 'Design'}

any_title_filter = ['sales']
all_title_filter = ['consu']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Sales Consultant'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Strategic Sales', 'Sales Management', 'Training', 'Customer Service', 'Channel Sales', 'Field Sales', 'Customer Relationship Management (CRM)', 'People Management', 'Sales', 'Business Development', 'Oral / Verbal Communication', 'Salesforce', 'Medical Terminology', 'Insurance', 'Business to Business (B2B) Sales', 'Technical Sales', 'Customer Relations', 'Account Sales', 'Account Management', 'Negotiation'}

any_title_filter = ['terri']
all_title_filter = ['sale']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Territory Sales'
#------------------------------------------------------------------------------------------------------
skills_filter = {'HTML', 'Project Management', 'Agile Software Development', 'Automation Scripting', 'Windows Programming / API', 'Software Development', 'Microsoft SQL Server', 'Java Script', 'Database Administration', 'Virtualization', 'HTML5', 'Cascading Style Sheets (CSS)', 'SQL', 'Product Development', 'Cloud Computing', 'System Integration', 'Application Development', 'Design', 'Data Analysis', 'Servicenow'}

any_title_filter = ['serv']
all_title_filter = ['now']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Servicenow Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Real Estate', 'Direct Marketing', 'Sales Management', 'Customer Relationship Management (CRM)', 'Customer Service', 'Sales', 'Negotiation'}

any_title_filter = ['real']
all_title_filter = ['estat']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Real Estate Sales'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Ruby on Rails', 'PHP', 'HTML', 'Laravel Web Framework', 'Node.js', 'React.js', 'ASP.NET Framework', 'Web Development', 'Java Script', 'Python', 'Wordpress', 'Vue.js', 'HTML5', 'Cascading Style Sheets (CSS)', 'SQL', 'MySQL', 'Java', 'Angular', 'C# Programming Language', '.NET'}

any_title_filter = ['web']
all_title_filter = ['dev']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Web Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Computer Aided Design (CAD)', 'Manufacturing Process Engineering', 'Autodesk AutoCAD', 'Engineering Design', 'Project Management', 'Microsoft Office', 'Quality Assurance / Quality Control', 'Production Support', 'Lean Manufacturing', 'Process Improvement', 'Problem Solving', 'People Management', 'Operations Management', 'Computer Aided Drafting & Design (CADD)', 'Well Production Engineering', 'Chemical Process Engineering', 'Production Management', 'Product Development', 'SolidWorks', 'Microsoft Excel'}

any_title_filter = ['production']
all_title_filter = ['engi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Production Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Git', 'iOS', 'Unit Testing', 'Software Architecture', 'Microsoft SQL Server', 'Firebase', 'Typescript', 'Software Development', 'Swift Programming Language', 'C++ Programming Language', 'Java Script', 'Apple Xcode IDE', 'Project Management', 'Mobile applications development', 'MacOS', 'flutter', 'Objective-C', 'Object Oriented Programming (OOP)', 'iOS SDK', 'Agile Software Development'}

any_title_filter = ['ios']
all_title_filter = ['deve']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'IOS Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Git', 'Maintenance', 'Software Test', 'Automation Scripting', 'C# Programming Language', 'Java', 'SQL', 'Software Development', 'Java Script', 'Regression Testing', 'Labview', 'verilog', '.NET', 'Test Planning', 'Selenium Automated Test Tool', 'Troubleshooting', 'Python', 'Test Automation', 'Apache JMeter', 'Agile Software Development'}

any_title_filter = ['autom']
all_title_filter = ['test']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Automation Tester'
#------------------------------------------------------------------------------------------------------
skills_filter = {'3D Rendering', 'Computer Aided Design (CAD)', 'Design', 'Technical Specification', 'Sales', 'Autodesk 3ds Max', 'ArchiCAD', 'Adobe Creative Suite', 'Project Coordinating', 'Autodesk AutoCAD', 'Project Management', 'Graphic Design', 'Computer Aided Drafting & Design (CADD)', 'Space Planning', 'Adobe Photoshop', 'Autodesk Revit Architecture', 'Microsoft Office', 'SketchUp', 'Adobe InDesign', 'Client Interaction'}

any_title_filter = ['interi']
all_title_filter = ['desi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Interior Designer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Marketing Communications', 'Documentation', 'Email Marketing', 'Editing', 'Writing Procedures', 'Content Management', 'Search Engine Optimization (SEO)', 'Technical Writing', 'Proposal Writing', 'Web Content Management', 'Social Media Marketing', 'Copywriting', 'Wordpress', 'Graphic Design', 'Blogging', 'Proofreading', 'Creative Writing', 'Microsoft Office', 'Online Research', 'English Language', 'Digital Marketing'}

any_title_filter = ['conte']
all_title_filter = ['wri']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Content Writer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Big Data Analytics', 'R', 'Data Analysis', 'Microsoft Excel', 'SAS', 'Tableau Software', 'Microsoft SQL Server', 'Alteryx Designer', 'SQL', 'Data Modeling', 'Machine Learning', 'Data Mining', 'Reporting', 'Microsoft Power BI', 'Data Entry', 'Python', 'VBA', 'Database Management', 'Microsoft Office', 'Data Visualization', 'Statistical Analysis'}

any_title_filter = ['data']
all_title_filter = ['anal']
not_title_filter = ['big','tech','engi']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Data Analyst'
#------------------------------------------------------------------------------------------------------
skills_filter = {'System Repair', 'Data Analysis', 'Maintenance', 'Systems Engineering', 'Information Technology (IT) Support', 'Manufacturing Process Engineering', 'Electronic Troubleshooting', 'Customer Service', 'Pro/Engineer', 'Project Management', 'Equipment Maintenance', 'Mechanical Tool Repair', 'Technical Services', 'Network Management / Administration', 'Troubleshooting', 'Programmable Logic Controllers (PLC) / Automation', 'Engineering Design', 'Technical Analysis', 'Systems Troubleshooting', 'Electronic Equipment Installation'}

any_title_filter = ['serv']
all_title_filter = ['engi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Service Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Salesforce', 'Pre-Sales', 'Data Analysis', 'Enterprise Solutions', 'System Architecture', 'Java', 'Software Architecture', 'SQL', 'Software Development', 'IT Consulting', 'Microsoft Azure', 'Project Management', 'Enterprise Application Integration', 'Network Architecture / Design', 'Business Analysis', 'Technical Sales', '.NET', 'Cloud Computing', 'J2EE', 'Amazon Web Services (AWS)'}

any_title_filter = ['solu']
all_title_filter = ['arch']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Solutions Architect'
#------------------------------------------------------------------------------------------------------
skills_filter = {'User Interface Design', 'Design', 'User Research', 'User Interface', 'User Experience (UX) Design', 'Web Design', 'Digital Design', 'Product Development', 'Adobe Illustrator', 'Information Architecture', 'Adobe Creative Suite', 'Adobe XD', 'Graphic Design', 'Prototyping', 'Figma', 'Adobe Photoshop', 'Problem Solving', 'e-Commerce', 'Wireframing', 'Interaction Design'}

any_title_filter = ['ui','ux']
all_title_filter = ['desi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Ui/Ux Designer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Project Management', 'Electronic Troubleshooting', 'Programmable Logic Controllers (PLC) / Automation', 'Field-Programmable Gate Array (FPGA)', 'Autodesk AutoCAD Electrical', 'C++ Programming Language', 'Board Design', 'Altium Designer', 'Python', 'Debug', 'Computer Aided Design (CAD)', 'Autodesk AutoCAD', 'Matlab', 'Systems Engineering', 'Maintenance', 'Autodesk Revit MEP', 'C Programming Language', 'Engineering Design', 'Software Development', 'Microsoft Office', 'Microsoft Excel'}

any_title_filter = ['elec']
all_title_filter = ['eng']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Electrical Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Content Management', 'HTML5', 'Design', 'Web Design', 'Java Script', 'Google Analytics', 'Board Design', 'Search Engine Optimization (SEO)', 'jQuery', 'Debug', 'Cascading Style Sheets (CSS)', 'Bootstrap', 'PHP', 'User Interface Design', 'HTML', 'Help Desk / Desktop Support', 'Troubleshooting', 'Windows Programming / API', 'Web Development', 'Wordpress', 'MySQL'}

any_title_filter = ['word']
all_title_filter = ['pre','deve']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Wordpress Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Project Management', 'Autodesk Inventor', 'ANSYS Simulation Software', 'CATIA', 'Product Development', 'HVAC System Design', 'SolidWorks', 'Finite Element Analysis (FEA)', 'Python', 'Autodesk AutoCAD', 'Computer Aided Design (CAD)', 'Matlab', 'Manufacturing Process Engineering', 'Maintenance', 'Microsoft Word', 'Pro/Engineer', 'Autodesk Revit MEP', 'Engineering Design', 'Microsoft Office', 'Microsoft Excel'}

any_title_filter = ['mech']
all_title_filter = ['eng']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Mechanical Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Project Management', 'Marketing Communications', 'Digital Marketing', 'Social Media Marketing', 'Sales Management', 'Strategic Marketing', 'Product Development', 'Strategic Planning', 'Product Management', 'Strategy', 'Budget Management', 'Product Marketing', 'Data Analysis', 'Event Management', 'Marketing Management', 'Branding', 'Sales', 'Media / Public Relations', 'Graphic Design'}

any_title_filter = ['brand']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Brand Management'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Firebase', 'Network Support', 'React Native', 'Java', 'Problem Solving', 'Figma', 'Android Software Development Kit (SDK)', 'Dart Programming Language', 'Java Script', 'Git', 'Python', 'Mobile applications development', 'RESTful Web Services', 'flutter', 'User Interface Design', 'Systems Troubleshooting', 'iOS', 'Software Development', 'REST', 'Japanese Language'}

any_title_filter = ['flutt']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Flutter Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Web Analytics', 'Content Management', 'Project Management', 'Web Content Management', 'Digital Marketing', 'Web Design', 'Social Media Marketing', 'Google Analytics', 'Search Engine Optimization (SEO)', 'Research Analysis', 'Technical Writing', 'HTML', 'Data Analysis', 'Account Management', 'Copywriting', 'Web Development', 'Wordpress', 'Microsoft Office', 'Search Engine Marketing (SEM)', 'Editing'}

any_title_filter = ['seo']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Search Engine Optimization (SEO)'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Business Analysis', 'Project Management', 'Training', 'Product Support', 'Production Support', 'Microsoft SQL Server', 'Information Technology (IT) Support', 'Applications', 'Application Development', 'Software Knowledge', 'Linux', 'SQL', 'Systems Troubleshooting', 'Help Desk / Desktop Support', 'Data Analysis', 'Troubleshooting', 'Customer Service', 'Software Development', 'Microsoft Office', 'Microsoft Excel'}

any_title_filter = ['appli']
all_title_filter = ['suppo']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Application Support'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Investment Management', 'Portfolio Management', 'Benefits & Compensation', 'Strategic Planning', 'Research Analysis', 'Facilitator', 'Client Interaction', 'Training Program Development', 'Valuation', 'Financial Modeling', 'Change Management', 'SQL', 'Data Analysis', 'Market Research', 'Operations Management', 'Presentations', 'Financial Analysis', 'Equity Management', 'Bloomberg Terminal', 'Microsoft Excel'}

any_title_filter = ['equit']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Equity Dealer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Algorithm Development', 'Firebase', 'Object Oriented Programming (OOP)', 'Swift Programming Language', 'Software Architecture', 'React Native', 'Java', 'Android Software Development Kit (SDK)', 'Java Script', 'Android Operating System Development', 'Git', 'Mobile applications development', 'RESTful Web Services', 'SQLite', 'Test Driven Development (TDD)', 'flutter', 'SQL', 'Kotlin Programming Language', 'iOS', 'Software Development'}

any_title_filter = ['andro']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Android Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Microsoft Azure', 'Windows Operating System General Use', 'Red Hat Enterprise Linux (RHEL)', 'Microsoft SQL Server', 'Windows PowerShell', 'Information Technology (IT) Support', 'Network Management / Administration', 'Microsoft Exchange', 'Salesforce', 'Virtualization', 'Linux', 'Microsoft Active Directory', 'SQL', 'Windows Support', 'VMware ESX', 'Windows Server 2012 R2', 'System Administration', 'Microsoft Office', 'Amazon Web Services (AWS)', 'IT Security & Infrastructure'}

any_title_filter = ['syst']
all_title_filter = ['adm']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'System Administrator'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Web Analytics', 'Project Management', 'Content Management', 'Marketing Communications', 'Web Content Management', 'Digital Marketing', 'Social Media Marketing', 'Google Analytics', 'Google Ad Words', 'Search Engine Optimization (SEO)', 'Strategic Marketing', 'Email Marketing', 'Paid Media Marketing', 'Data Analysis', 'Marketing Management', 'Online Marketing', 'Web Development', 'Advertising', 'Graphic Design', 'Search Engine Marketing (SEM)'}

any_title_filter = ['digit']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Digital Marketing'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Business Analysis', 'System Architecture', 'Database', 'Software Architecture', 'C# Programming Language', 'Java Script', 'Microsoft SQL Server', 'Oracle Application Express', 'PL/SQL', 'Oracle SQL', 'Shell Scripting', 'Python', 'IT Management', 'SQL', 'Data Modeling', 'Data Analysis', 'IT Consulting', '.NET', 'MySQL', 'Enterprise Resource Planning (ERP)'}

any_title_filter = ['erp']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Enterprise Resource Planning (ERP) Developer'
#------------------------------------------------------------------------------------------------------


In [117]:
skills_filter = {'Business Analysis', 'System Architecture', 'Database', 'Software Architecture', 'C# Programming Language', 'Java Script', 'Microsoft SQL Server', 'Oracle Application Express', 'PL/SQL', 'Oracle SQL', 'Shell Scripting', 'Python', 'IT Management', 'SQL', 'Data Modeling', 'Data Analysis', 'IT Consulting', '.NET', 'MySQL', 'Enterprise Resource Planning (ERP)'}

any_title_filter = ['erp']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

# df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Enterprise Resource Planning (ERP) Developer'
# for i in df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not]['job_title']:
#     print(i)
df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not]

,job_title,company_name,experience,salary,location,description,posted_on,current_date,link,sal_range,...,skill35,skill36,skill37,skill38,skill39,skill40,mapped_average_sal,mapped_job_title,mapped_job_title_1,combined_skills
499,Business Analyst ( ERP Admin Module ),laminaar aviation infotech,3 - 5 years,3-6,"Chennai,Chennai",Areas of Specialization: Requirements Specific...,30+ days ago,2024-04-02,https://www.naukri.com/job-listings-business-a...,3to6,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Business Analyst Erp Admin Module,"Administration,Enterprise Resource Planning (E..."
2599,Data Analysis & Interpretation-Application Dev...,accenture,3 - 4 years,3-6,Chennai,Project Role :Application Developer\n\nProject...,1 day ago,2024-04-02,https://www.naukri.com/job-listings-data-analy...,3to6,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Data Analysis Interpretation Application Devel...,"Data Analysis,SQL,Data Modeling,Snowflake Clou..."
2768,Enterprise Architect,ingram micro,3 - 5 years,3-6,"Chennai,Chennai",Oversee the design and implementation of the a...,14 days ago,2024-04-02,https://www.naukri.com/job-listings-enterprise...,3to6,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Enterprise Architect,"Microsoft Visio,SQL"
6248,GN - Strategy - MC - CFO EV - Enterprise Perfo...,accenture,3 - 5 years,3-6,Hyderabad,GN - Strategy - MC - CFO EV - Enterprise Perfo...,29 days ago,2024-04-03,https://www.naukri.com/job-listings-gn-strateg...,3to6,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Gn Strategy Mc Cfo Ev Enterprise Performance M...,"Accounting,Oral / Verbal Communication,SQL,C# ..."
8119,ERP Test Automation Specialist,coders brain pvt ltd,2 - 5 years,3-6,"Maheshtala,Chennai,Delhi,Hyderabad,Chennai,Che...",Functional Area:\n\nInformation Systems\n\nM...,30+ days ago,2024-04-03,https://www.naukri.com/job-listings-erp-test-a...,3to6,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Erp Test Automation Specialist,"Enterprise Resource Planning (ERP),Automation ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51119,Associate Director-Enterprise Applications(ERP...,allen digital,10 - 19 years,50-75,"Chennai,Bangalore",Qualifications and Skills:\n\nProven experienc...,7 days ago,2024-04-03,https://www.naukri.com/job-listings-associate-...,50to75,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Associate Director Enterprise Applications Erp...,"Applications,Human Resources (HR),Enterprise R..."
51176,GN - Strategy - MC - CFO EV - Enterprise Perfo...,accenture,6 - 9 years,50-75,Hyderabad,GN - Strategy - MC - CFO EV - Enterprise Perfo...,24 days ago,2024-04-03,https://www.naukri.com/job-listings-gn-strateg...,50to75,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Gn Strategy Mc Cfo Ev Enterprise Performance M...,"Project Management,Employee Relations,Employee..."
51221,MC - CFO EV - Enterprise Performance Managemen...,accenture,12 - 14 years,50-75,Chennai,Enterprise Performance Senior Manager or Enter...,30+ days ago,2024-04-03,https://www.naukri.com/job-listings-mc-cfo-ev-...,50to75,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Mc Cfo Ev Enterprise Performance Management Ep...,"Program Management,Performance Management,Proj..."
51295,Specialist IT Enterprise Data Assets - Data En...,eaton technologies,12 - 15 years,50-75,Chennai,"What you’ll do:\nIn Eaton, making our work exc...",9 days ago,2024-04-03,https://www.naukri.com/job-listings-specialist...,50to75,...,None,None,None,None,None,None,0,Enterprise Resource Planning (ERP) Developer,Specialist It Enterprise Assets Data Engineer,"Electronic Medical Records (EMR),IT Management..."


In [118]:
df[df['mapped_job_title'] == 'Admission Counsellor']

,job_title,company_name,experience,salary,location,description,posted_on,current_date,link,sal_range,...,skill35,skill36,skill37,skill38,skill39,skill40,mapped_average_sal,mapped_job_title,mapped_job_title_1,combined_skills
1287,Admission Counsellor,upgrad,1 - 5 years,4-6 Lacs P.A.,"Chennai,Bangalore",Role & responsibilities\nAct as a mentor and g...,11 days ago,2024-04-02,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,5.0,Admission Counsellor,Admission Counsellor,Counseling
2535,Admission Counsellor,arbor academy,0 - 5 years,1.5-6.5 Lacs P.A.,Chennai,Role & responsibilities\n\n1.Engage with stude...,5 days ago,2024-04-02,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,4.0,Admission Counsellor,Admission Counsellor,"Counseling,Counseling,Counseling,Counseling"
3020,Admission Counsellor,upgrad,0 - 5 years,4-6 Lacs P.A.,"Noida,Pune,Chennai","Looking For Gurgaon , Noida & Hyderabad Candid...",11 days ago,2024-04-02,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,5.0,Admission Counsellor,Admission Counsellor,"Counseling,Sales,Counseling"
3087,Hiring For Admission Counselor's and Senior Ad...,upgrad,1 - 6 years,4-5.5 Lacs P.A.,"Noida,Pune,Hyderabad,Hyderabad,Chennai,Pune",Role & responsibilities\n\nThe ideal candidate...,16 days ago,2024-04-03,https://www.naukri.com/job-listings-hiring-cou...,3to6,...,None,None,None,None,None,None,4.75,Admission Counsellor,Senior Admission Counselor,"Inside Sales,Counseling,Oral / Verbal Communic..."
3107,Admission Counsellor,upgrad,4 - 6 years,3-6 Lacs P.A.,"Gurugram,haryana,Chennai,Faridabad,Pune",Job Scope\n\n\nThe ideal candidate for this ro...,11 days ago,2024-04-03,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,4.5,Admission Counsellor,Admission Counsellor,"Sales,Counseling,Counseling"
6139,Admission Counsellor,upgrad,2 - 6 years,4-7 Lacs P.A.,"Mumbai,Bangalore",The ideal candidate for this role is that you ...,15 days ago,2024-04-03,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,5.5,Admission Counsellor,Admission Counsellor,"Counseling,Counseling,Counseling,Oral / Verbal..."
6380,Admission Counsellor,upgrad,1 - 6 years,3-7 Lacs P.A.,"Noida,Pune,Hyderabad,telangana,Chennai",Job Description:\nThe ideal candidate for this...,23 days ago,2024-04-03,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,5.0,Admission Counsellor,Admission Counsellor,"Sales,Sales,Inside Sales,Sales,Counseling"
12353,Admission Counsellor,indian school of business managmenet and admin...,0 - 3 years,"50,000-3 Lacs P.A.","Hyderabad,Hyderabad",Company: Indian School of Business Management ...,22 days ago,2024-04-04,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,1.75,Admission Counsellor,Admission Counsellor,"Sales,Counseling,Sales,Sales,Counseling,Counse..."
14743,Admission Counsellor,emerald group of schools,2 - 3 years,3-6,Chennai,Any graduate/post graduate with 2-3 years expe...,6 days ago,2024-04-04,https://www.naukri.com/job-listings-admission-...,3to6,...,None,None,None,None,None,None,0,Admission Counsellor,Admission Counsellor,"Administration,Office Administration,Sales,Cou..."
18744,Admission Counsellor,arbor academy,0 - 5 years,1.5-6.5 Lacs P.A.,Chennai,Role & responsibilities\n\n1.Engage with stude...,7 days ago,2024-04-04,https://www.naukri.com/job-listings-admission-...,6to10,...,None,None,None,None,None,None,4.0,Admission Counsellor,Admission Counsellor,"Counseling,Counseling,Counseling,Counseling"


In [119]:
len(df['mapped_job_title'].unique())

81

In [120]:
for i in df['mapped_job_title'].unique():
    if '' in i.lower():
        print(i)

unclassified
Software Engineer
Sales Officer
Content Writer
Account Manager
Human Resources
Graphic Designer
Android Developer
Php Developer
Digital Marketing
HR Generalist
Sap Consultant
Mechanical Engineer
Territory Sales
Business Development
Servicenow Developer
Design Engineer
Search Engine Optimization (SEO)
Customer Support
Electrical Engineer
Application Support
Technical Architect
Marketing Manager
Service Engineer
BFSI, Investments & Trading
Devops Engineer
Data Engineer
Sales Consultant
Recruitment & Talent Acquisition
Production & Manufacturing
Relationship Manager
Mobile / App Developer
Field Sales Executive
Recruiter
Desktop Support Engineer
Java Developer
Customer Care
Data Analyst
Full Stack Developement
Front End Developer
System Administrator
Tele Sales
Python Developer
Automation Tester
Dot Net Developer
Tech Lead
Data Scientist
Business Analyst
IT Infrastructure Services
Inside Sales
IT & Information Security
Enterprise Resource Planning (ERP) Developer
IT Support
Sa

In [121]:
l = '''
Business Development Executive
Sales Executive
Business Development Manager
Sales Manager
Relationship Manager
Graphic Designer
Java Developer
Field Sales Executive
Digital Marketing Executive
Dot Net Developer
Area Sales Manager
Inside Sales Executive
Marketing Executive
Java Full Stack Developer
HR Recruiter
Business Development Associate
Marketing Manager
Tele Sales Executive
Sales Engineer
Digital Marketing Manager
Business Analyst
Sales Officer
Customer Support Executive
Fullstack Developer
Customer Care Executive
Sales Coordinator
Hr Executive And Recruiter
Software Engineer
Devops Engineer
HR Manager
Senior Java Developer
Data Engineer
HR Executive
Interior Designer
Assistant Sales Manager
Software Developer
Regional Sales Manager
Academic Counsellor
Branch Manager
Recruiter
Design Engineer
Territory Sales Manager
Talent Acquisition Specialist
Front End Developer
Digital Marketing Specialist
Desktop Support Engineer
Network Engineer
Php Developer
Tele Caller
Inside Sales Specialist
Social Media Manager
Telecalling Executive
Senior Software Engineer
Admission Counsellor
Angular Developer
Data Scientist
Key Accounts Manager
Hr Executive And Generalist
Equity Dealer
React JS Developer
Python Developer
Data Analyst
Senior Sales Executive
Full Stack Developer
Service Engineer
iOS Developer
Web Developer
Bancassurance Manager
Human Resource Manager
Android Developer
Assistant Manager
Salesforce Developer
Bancassurance Sales Manager
Content Writer
Sap Abap Consultant
Technical Lead
Staff Nurse
React Native Developer
Social Media Executive
HR Generalist
Human Resource Executive
Azure Data Engineer
It Recruiter
Flutter Developer
Talent Acquisition Executive
Us It Recruiter
Mechanical Engineer
Digital Marketing Intern
Architect
Sales Associate
Wordpress Developer
Retail Sales Executive
International Sales Executive
Sales Representative
Technical Support Engineer
System Administrator
Seo Executive
Node Js Developer
Electrical Engineer
HR Intern
Servicenow Developer
Performance Marketing Manager
Field Sales Officer
Lead Generation Executive
Python Software Developer
Production Engineer
Dot Net Fullstack Developer
Medical Representative
Senior Graphic Designer
Customer Service Executive
Executive Assistant
Bench Sales Recruiter
Agency Development Manager
Production Supervisor
Java Back End Developer
Real Estate Sales Executive
Field Sales Executive/Senior Executive Client Acquisition
Presales Executive
UI/UX Designer
Human Resource Intern
Production Manager
B2B Sales Executive
Education Counsellor
Walk-in - PhonePe is Hiring II Field Sales Officer II CTC Upto 5.5 LPA
PHP Developer
Automation Test Engineer
IT Recruiter
Executive/ Senior Executive Business Development
Agency Manager
Virtual Relationship Manager
Team Lead
Branch Sales Manager
Tech Lead
Tender Executive
Sales And Marketing Executive
Ui Ux Developer
Medical Sales Representative
Maintenance Engineer
Senior Sales Manager
Inside Sales Associate
Technical Support Executive
Pharmacist
Seo Specialist
Technical Architect
Sap Abap Developer
Laravel Developer
Field Sales Manager
Mis Executive
Sales And Marketing Manager
Inside Sales Manager
Business Development Intern
Sr. Software Engineer
Field Sales Executive / Field Sales Officer / Field Officer
Java Architect
Real Estate Sales Manager
Corporate Sales Manager
Sales Intern
Senior Business Analyst
Management Trainee
Mechanical Design Engineer
Sales Consultant
Senior Relationship Manager
Customer Relationship Manager
Power Bi Developer
Sap Fico Consultant
Senior Data Engineer
Application Support Engineer
Territory Sales Executive
Sales Development Representative
Project Manager
Technical Sales Executive
PhonePe is Hiring II Field Sales Officer II CTC Upto 5.5 LPA
Lead Generation Specialist
Telesales Executive
Golang Developer
Database Administrator
Solution Architect
It Support Engineer
Software Test Engineer
.Net Developer
Automation Tester
Brand Manager
Senior Python Developer
Technical Business Analyst
Recruiter Specialist
Architect and Interior Designer
Business Development Manager Life Insurance
Medical Coder
Agency Manager / Sales Manager / Relationship Manager
Engineering Manager
Sales Head
Back End Developer
Business Development Officer
Java Lead
Backend Developer
'''
l = l.strip().split('\n')
l = set(l)

malfun = ['full stack developer','front end developer','back end developer','social media marketing','marketing','marketing and communication','lead generation / qualification','Digital Marketing Executive',
          'business development','Senior Software Engineer','']
for i in l:
    i = i.replace('amp;','').replace(' - other','').lower()
    if all(word.lower() not in i for word in df['mapped_job_title'].unique()): #and all(word.lower() not in i for word in malfun):
        print(i)

bancassurance manager
human resource executive
hr manager
education counsellor
.net developer
sales representative
mis executive
production supervisor
sales associate
architect
agency development manager


software test engineer
social media executive
assistant sales manager
telecalling executive
backend developer
staff nurse
sales executive
key accounts manager
software developer
medical sales representative
human resource intern
regional sales manager
tele caller
management trainee
technical lead
maintenance engineer
java lead
seo specialist
sales intern
marketing executive
technical support executive
senior sales manager
power bi developer
golang developer
agency manager
team lead
angular developer
automation test engineer
pharmacist
talent acquisition executive
medical representative
executive assistant
presales executive
react native developer
project manager
java full stack developer
bancassurance sales manager
customer service executive
b2b sales executive
ui ux developer
technical support engineer
laravel developer
python software developer
sap abap developer
fullstack developer
sales manager
telesales executive
full stack developer
sales development representative
node js develo

In [122]:
skillvalcount = {}

def valcount(lis):
    for i in lis:
        if i in skills_edited:
            if i in skillvalcount:
                skillvalcount[i] += 1
            else:
                skillvalcount[i] = 1

for index, val in df[skilcol].iterrows():
    val = val.to_list()
    valcount(val)

In [123]:
skillvalcountdf = pd.DataFrame.from_dict(skillvalcount, orient='index',columns=['count'])
skillvalcountdf.index.name = 'skills from payscale'
skillvalcountdf.to_excel('mapped_pay_scale_skills.xlsx')

In [124]:
pd.DataFrame.from_dict(skills_edited, orient='index').T.to_excel('mapped_pay_scale_skills.xlsx',index=False)

In [125]:
# for k,v in skills_edited.items():
#     skills_edited[k]=len(v)

In [126]:
# p = pd.DataFrame.from_dict(skills_edited, orient='index',columns=['counts'])
# p.index.name = 'skills'
# p.to_excel('payscale_skills_mapped_counts.xlsx')

In [127]:
df['combined_skills'] = df[skilcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)

In [128]:
f = []
for k,v in df[df['mapped_job_title']=='unclassified']['job_title'].value_counts().items():
    # f.append(list(k,v))
    l = []
    l.append(k)
    l.append(v)
    f.append(l)
fdf = pd.DataFrame(f,columns = ['role','count'])

fdf.to_excel('unmapedroles.xlsx',index = False)

In [129]:
cdf = df[df['mapped_job_title']=='unclassified']['job_title']
cdf

0                                               Pharmacist
1                                               Pharmacist
4                                  BOAT APPRENTICE TRAINEE
5                               Customer Service Executive
6                                              Optometrist
                               ...                        
64215    Associate, Client Onboarding / Transitions / C...
64216              Associate Professional, Presale Support
64217              Associate Professional, Presale Support
64218              Associate Professional, Presale Support
64219              Associate Professional, Presale Support
Name: job_title, Length: 39096, dtype: object

In [130]:
# print(len(df['role'].unique()))
# df.loc[df['role'].isna(),'role'] = df['mapped_job_title']
# df['mapped_job_title'] = df['role'] 
# df.loc[(df['mapped_job_title'].isna()) | (df['mapped_job_title']=='Nan'),'mapped_job_title'] = 'unclassified'
#123321

In [131]:
df['mapped_job_title'].unique()

array(['unclassified', 'Software Engineer', 'Sales Officer',
       'Content Writer', 'Account Manager', 'Human Resources',
       'Graphic Designer', 'Android Developer', 'Php Developer',
       'Digital Marketing', 'HR Generalist', 'Sap Consultant',
       'Mechanical Engineer', 'Territory Sales', 'Business Development',
       'Servicenow Developer', 'Design Engineer',
       'Search Engine Optimization (SEO)', 'Customer Support',
       'Electrical Engineer', 'Application Support',
       'Technical Architect', 'Marketing Manager', 'Service Engineer',
       'BFSI, Investments & Trading', 'Devops Engineer', 'Data Engineer',
       'Sales Consultant', 'Recruitment & Talent Acquisition',
       'Production & Manufacturing', 'Relationship Manager',
       'Mobile / App Developer', 'Field Sales Executive', 'Recruiter',
       'Desktop Support Engineer', 'Java Developer', 'Customer Care',
       'Data Analyst', 'Full Stack Developement', 'Front End Developer',
       'System Administrat

In [132]:
#asd123

In [133]:
host = "54.79.186.42"
user = "root"
password = "secondstorey"
database = "equipay_partners"

# Get the current date for backup filename
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
backup_filename = f"PayPulse_backup_{current_date}.csv"

# Backup the current table (seek_extract) to a CSV file
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
connection = mysql.connector.connect(
host=host,
user=user,
password=password,
database=database
)

cursor = connection.cursor()
cursor.execute("SELECT * FROM paypulse_profiles")
rows = cursor.fetchall()
column_names = [i[0] for i in cursor.description]

backup_df = pd.DataFrame(rows, columns=column_names)

backup_df.to_csv(backup_filename, index=False)
connection.close()

####################################################################################
df.columns = df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()
backup_df.columns = backup_df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()
df.rename(columns={'current_date':'extracted_on'},inplace=True)
df = pd.concat([backup_df,df],ignore_index=True)

In [ ]:
subset = ['job_title', 'company_name', 'experience', 'salary', 'location',
       'skill1', 'skill2', 'skill3', 'skill4','skill5', 'skill6', 'skill7', 'skill8', 'role', 'industry_type',
       'department', 'employment_type', 'role_category', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills', 'salmin',
       'salmax', 'ori_experience', 'minexp', 'maxexp', 'title_id','avg_experience']
df.drop_duplicates(subset= subset, inplace=True)

In [134]:
renjidict = {}

for i in df['mapped_job_title'].unique():
    filtdf = df[df['mapped_job_title'] == i][skilcol]
    for index, j in filtdf.iterrows():        
        for k in j:
            if (k != '') and isinstance(k, str):
                if not (len(k)<2):        
                    if i in renjidict:
                        renjidict[i].add(k)
                    else:
                        renjidict[i] = set(k)

In [135]:
for key, value_set in renjidict.items():
    filtered_set = {word for word in value_set if len(word) > 2 and not word.isdigit()}
    renjidict[key] = filtered_set

In [136]:
allskills = set()
for col in renjidict:
    allskills.update(renjidict[col])
len(allskills)

479

In [137]:
value_counts = {}
for col in renjidict:
    value_counts[col] = len(renjidict[col])
converted = pd.DataFrame.from_dict(value_counts, orient='index')
converted.to_excel('skill_value_counts.xlsx')

In [138]:
converted

,0
Devops Engineer,122
unclassified,468
Customer Support,87
Desktop Support Engineer,51
Software Engineer,276
...,...
Direct Sales Executive,58
Social Media Marketing,67
Interior Designer,42
Production Engineer,47


In [139]:
sum = 0
for i in value_counts:
    sum += value_counts[i] 
print(sum)
df

10221


,id,job_title,company_name,experience,salary,location,posted_on,extracted_on,skill1,skill2,...,skill31,skill32,skill33,skill34,skill35,skill36,skill37,skill38,skill39,skill40
0,1.0,Aws Devops Engineer,tech mahindra,5-8,16-22.5 Lacs PA,"Hyderabad, telangana,Chennai,Bangalore,Gurugra...",1 Day Ago,2023-08-22,Terraform,Development Operations (DevOps),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Hiring B. Tech Freshers || Hyderabad || Excell...,hcltech,0-2,"50,000-2.5 Lacs PA","Hyderabad, telangana",1 Day Ago,2023-08-22,Oral / Verbal Communication,HTML,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Day shift - Hindi Voice Process - 3L CTC,oxilum technologies,0-2,2.5-3 Lacs PA,Chennai,1 Day Ago,2023-08-22,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Great Salaries For Tech Support in Bangalore -...,jobshop,8-11,4-5.5 Lacs PA,Chennai,Just Now,2023-08-22,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Customer Support Executive,outlook hr solutions,0-2,2.25-4 Lacs PA,Hyderabad,1 Day Ago,2023-08-22,None,Oral / Verbal Communication,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271900,NaN,"Associate, Client Onboarding / Transitions / C...",bny mellon,1 - 4 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271901,NaN,"Associate Professional, Presale Support",ingram micro,2 - 5 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271902,NaN,"Associate Professional, Presale Support",ingram micro,2 - 5 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271903,NaN,"Associate Professional, Presale Support",ingram micro,2 - 5 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None


In [140]:
df.to_csv('Checkpoint2.csv',index=False)

# value counts of payscale skills

In [141]:
renjidictvc = {} 

for i in df['mapped_job_title'].unique():
    if i and isinstance(i, str) and (i.lower() != 'nan') and (i != ''):
        filtdf = df[df['mapped_job_title'] == i][skilcol]
        skill_counts = {}
        for index, j in filtdf.iterrows():
            for k in j:
                if (k != '') and isinstance(k, str) and k.lower() != 'nan':
                    if k in skill_counts:
                        skill_counts[k] += 1
                    else:
                        skill_counts[k] = 1
    else:
        continue
    renjidictvc[i] = skill_counts

print(renjidictvc)


{'Devops Engineer': {'Terraform': 377, 'Development Operations (DevOps)': 1285, 'Kubernetes': 489, 'Amazon Web Services (AWS)': 465, 'Google Cloud Platform (GCP)': 140, 'Continuous Integration (CI)': 176, 'Jenkins': 283, 'Python': 286, 'Machine Learning': 10, 'GIT': 321, 'Shell Scripting': 91, 'Network Security Management': 5, 'Automation Scripting': 194, 'Linux': 222, 'Troubleshooting': 67, 'Docker': 311, 'XML': 12, 'Administration': 25, 'Go (Golang) Programming Language': 19, 'Microsoft Azure': 305, 'PostgreSQL': 25, 'Apache Hadoop': 3, 'Windows PowerShell': 67, 'Apache Kafka': 9, 'SQL': 38, 'Telemetry': 1, 'Spark': 4, 'Database': 8, 'ETL (Extract, transform, load)': 14, 'MySQL': 29, 'Leadership': 2, 'Java Script': 106, 'Enterprise Resource Planning (ERP)': 1, 'Debug': 20, 'C++ Programming Language': 14, 'Hiring': 3, 'Accounting': 1, 'Windows Operating System General Use': 1, 'MongoDB': 17, 'React.js': 3, 'Windows Server': 3, '.NET': 9, 'Virtualization': 4, 'Microsoft SQL Server': 8,

In [142]:
for i in renjidict.keys():
    if 'unclass' in i:
        print(i)

unclassified


In [143]:
pd.DataFrame.from_dict(renjidictvc, orient='index').T.to_excel('payscale_skill_value_counts.xlsx')

In [144]:
payscale_skilldf = pd.read_excel('payscale_skill_value_counts.xlsx')

In [145]:
columns_to_sum = payscale_skilldf.columns.drop(['unclassified','Unnamed: 0'])
payscale_skilldf['count'] = payscale_skilldf[columns_to_sum].sum(axis=1)

In [146]:
payscale_skilldf = payscale_skilldf.sort_values(by='count', ascending=False)

In [147]:
payscale_skilldf

,Unnamed: 0,Devops Engineer,unclassified,Software Engineer,Java Developer,Technical Architect,Full Stack Developement,Python Developer,Tech Lead,Data Engineer,...,Academic Counsellor,Real Estate Sales,Sales Support & Operations,Social Media Marketing,Retail Sales,Customer Care,Medical Coder,Interior Designer,Medical Biller,count
131,Sales,NaN,40904.0,35.0,6.0,117.0,6.0,NaN,38.0,3.0,...,415.0,1116.0,8.0,15.0,719.0,125.0,NaN,34.0,NaN,47955.0
31,Java Script,106.0,6547.0,2286.0,3474.0,1256.0,1152.0,174.0,508.0,218.0,...,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,23394.0
66,Oral / Verbal Communication,26.0,14960.0,525.0,39.0,701.0,87.0,11.0,84.0,98.0,...,95.0,97.0,NaN,38.0,31.0,201.0,6.0,24.0,8.0,14898.0
155,Human Resources (HR),NaN,2164.0,9.0,NaN,3.0,NaN,NaN,3.0,6.0,...,6.0,32.0,NaN,NaN,1.0,4.0,NaN,NaN,NaN,11848.0
46,Applications,16.0,6449.0,480.0,219.0,442.0,134.0,62.0,61.0,45.0,...,NaN,3.0,NaN,2.0,NaN,NaN,NaN,1.0,2.0,11748.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Writing Procedures,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
432,Academic Advising,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
443,Rehabilitation,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
275,Emergency/Trauma,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [148]:
# payscale_skilldf = payscale_skilldf[~(payscale_skilldf['Unnamed: 0'].str.islower()) & (payscale_skilldf['count'] < 500)]

In [149]:
roles = list(payscale_skilldf.columns)
for i in ['Unnamed: 0','count','unclassified']:
    try:
        roles.remove(i)
    except:
        continue

In [150]:
for i in range(len(roles)):
    if 'acc' in roles[i].lower():
        print(i,roles[i])

47 Account Manager


In [151]:
top_skills_data = {}
for i in range(len(roles)):
    temptop = payscale_skilldf.sort_values(by=roles[i], ascending=False)
    top_skills_data[roles[i]] = temptop['Unnamed: 0'].head(20).tolist()
    top_skills_data[f'{roles[i]}_count'] = temptop[roles[i]].head(20).tolist()
# Create a DataFrame from the dictionary 
topskillsdf = pd.DataFrame(top_skills_data)

In [152]:
price_a_job_titles = pd.DataFrame([title for title in topskillsdf if '_count' not in title],columns = ['titles'])
price_a_job_titles = price_a_job_titles.sort_values('titles').reset_index(drop=True)
price_a_job_titles['id'] = price_a_job_titles.index + 1
price_a_job_titles = price_a_job_titles.reindex(columns=['id','titles'])

In [153]:
checkset = set(price_a_job_titles['titles'])

In [154]:
# df = df[df['mapped_job_title'].apply(lambda x: x in checkset)]

In [155]:
len(df['mapped_job_title'].unique())

82

In [156]:
df

,id,job_title,company_name,experience,salary,location,posted_on,extracted_on,skill1,skill2,...,skill31,skill32,skill33,skill34,skill35,skill36,skill37,skill38,skill39,skill40
0,1.0,Aws Devops Engineer,tech mahindra,5-8,16-22.5 Lacs PA,"Hyderabad, telangana,Chennai,Bangalore,Gurugra...",1 Day Ago,2023-08-22,Terraform,Development Operations (DevOps),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Hiring B. Tech Freshers || Hyderabad || Excell...,hcltech,0-2,"50,000-2.5 Lacs PA","Hyderabad, telangana",1 Day Ago,2023-08-22,Oral / Verbal Communication,HTML,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Day shift - Hindi Voice Process - 3L CTC,oxilum technologies,0-2,2.5-3 Lacs PA,Chennai,1 Day Ago,2023-08-22,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Great Salaries For Tech Support in Bangalore -...,jobshop,8-11,4-5.5 Lacs PA,Chennai,Just Now,2023-08-22,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Customer Support Executive,outlook hr solutions,0-2,2.25-4 Lacs PA,Hyderabad,1 Day Ago,2023-08-22,None,Oral / Verbal Communication,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271900,NaN,"Associate, Client Onboarding / Transitions / C...",bny mellon,1 - 4 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271901,NaN,"Associate Professional, Presale Support",ingram micro,2 - 5 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271902,NaN,"Associate Professional, Presale Support",ingram micro,2 - 5 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271903,NaN,"Associate Professional, Presale Support",ingram micro,2 - 5 years,0-3,Chennai,30+ days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None


In [157]:
price_a_job_titles_dict = price_a_job_titles.set_index('titles')['id'].to_dict()
price_a_job_titles_dict.update({'unclassified':0})
df['title_id'] = df['mapped_job_title'].replace(price_a_job_titles_dict)

In [158]:
# table name : price_a_job_topskills  -   columns :  id, title_id , skill
price_a_job_topskills_dict = []
for col in [title for title in topskillsdf if '_count' not in title]:
    for topskill in topskillsdf[col].values:
        price_a_job_topskills_dict.append({'skill':topskill, 'title':col})

price_a_job_topskills = pd.DataFrame(price_a_job_topskills_dict)
price_a_job_topskills = price_a_job_topskills.sort_values('skill').reset_index(drop=True)
price_a_job_topskills['id'] = price_a_job_topskills.index + 1
price_a_job_topskills = price_a_job_topskills.reindex(columns=['id','title','skill'])
price_a_job_topskills = price_a_job_topskills.rename(columns={'title':'title_id'})
price_a_job_topskills['title_id'] = price_a_job_topskills['title_id'].replace(price_a_job_titles_dict)

In [159]:
df.columns

Index(['id', 'job_title', 'company_name', 'experience', 'salary', 'location',
       'posted_on', 'extracted_on', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'role', 'industry_type',
       'department', 'employment_type', 'role_category', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills', 'salmin',
       'salmax', 'ori_experience', 'minexp', 'maxexp', 'title_id',
       'avg_experience', 'description', 'link', 'sal_range', 'skill9',
       'skill10', 'skill11', 'skill12', 'skill13', 'skill14', 'skill15',
       'skill16', 'skill17', 'skill18', 'skill19', 'skill20', 'skill21',
       'skill22', 'skill23', 'skill24', 'skill25', 'skill26', 'skill27',
       'skill28', 'skill29', 'skill30', 'skill31', 'skill32', 'skill33',
       'skill34', 'skill35', 'skill36', 'skill37', 'skill38', 'skill39',
       'skill40'],
      dtype='object')

In [160]:
df[['salmin', 'salmax']] = 0

In [161]:
# # Convert 'avg_experience' to numeric, converting non-numeric values to NaN
# df['avg_experience'] = pd.to_numeric(df['avg_experience'], errors='coerce')
# # Now convert to float
# df['avg_experience'] = df['avg_experience'].astype(float)


In [162]:
df['title_id'].unique()

array([22,  0, 16, 21, 72,  6, 60, 13, 79, 42, 66, 81, 40, 69, 31, 76, 17,
       35, 44, 20, 32, 54, 57, 77, 75, 18,  8, 25, 55, 26,  7, 45, 39, 51,
       34, 80, 63, 67, 59, 74, 15, 58, 68, 48, 78, 19, 10, 49,  2, 29, 64,
       38, 33, 27,  9,  3, 71, 14, 50, 46,  1, 23, 11, 30, 61, 12, 43, 47,
       28, 37, 53, 73,  5, 62, 36, 56,  4, 24, 70, 41, 52, 65],
      dtype=int64)

In [163]:
df.to_csv('checkpoint.csv',index=False)

In [164]:
# df = pd.read_csv('checkpoint.csv')#------------------------------------------------------------------------------------------------------

In [165]:
def extract_min_max(salary_str):
    if isinstance(salary_str, str):
        if '$' in salary_str:
            salary_min = None
            salary_max = None 
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        l = []
        salary_str = salary_str.replace('Less than', '')
        
        if '/month' in salary_str:
            salary_str = salary_str.replace('/month', '').replace(',', '')
            salary_min = None
            salary_max = None 
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        try:
            a = salary_str.split('-')
            if a[0] == a[1]:
                salary_str = a[0]
        except:
            pass
            
        l = list(salary_str)
        
        if ('-' not in l) and all(ltr not in l for ltr in ['P','C','L']):
            salary_min = None
            salary_max = None
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        if 'L' in l and 'C' in l:    
            ind = l.index('L')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',', '') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                if None in l:
                    return pd.Series({'salmin': None, 'salmax': None})
                l = [float(i)/100000 if len(i) > 4 else float(i) for i in l]
                l[1] = l[1]*100
                salary_min = l[0]
                salary_max = l[1] 
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        elif 'L' in l:    
            ind = l.index('L')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',', '') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                if None in l:
                    return pd.Series({'salmin': None, 'salmax': None})
                l = [float(i)/100000 if len(i) > 4 else float(i) for i in l]
                salary_min = l[0]
                salary_max = l[1] 
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})
            else:
                salary_min = None
                salary_max = None
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})

        elif 'C' in l:
            ind = l.index('C')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',', '') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                if None in l:
                    return pd.Series({'salmin': None, 'salmax': None})
                l = [float(i)/10000000 if len(i) > 4 else float(i)*100 for i in l]
                salary_min = l[0]
                salary_max = l[1] 
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})
            else:
                salary_min = None
                salary_max = None
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})

        elif 'P' in l:
            salary_min = None
            salary_max = None
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        elif ('-' in l):
            l = ''.join(l).strip()
            l = l.split('-')
            salary_min = l[0]
            salary_max = l[1]
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        elif all(ltr not in l for ltr in ['P','C','L']):
            salary_min = None
            salary_max = None
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
    
    return pd.Series({'salmin': None, 'salmax': None})

df[['salmin', 'salmax']] = df['salary'].apply(extract_min_max)

In [166]:
df['salmin'] = pd.to_numeric(df['salmin'],errors='coerce')
df['salmax'] = pd.to_numeric(df['salmax'],errors='coerce')

In [167]:
df[df['salmin'].isna()] #check if count is more

,id,job_title,company_name,experience,salary,location,posted_on,extracted_on,skill1,skill2,...,skill31,skill32,skill33,skill34,skill35,skill36,skill37,skill38,skill39,skill40
208189,NaN,Software Engineer Jobs In New Delhi,n/a,N/A,N/A,n/a,N/A,2024-04-02,None,None,...,None,None,None,None,None,None,None,None,None,None
208339,NaN,Software Engineer Jobs In New Delhi,n/a,N/A,N/A,n/a,N/A,2024-04-02,None,None,...,None,None,None,None,None,None,None,None,None,None
208409,NaN,Developer Jobs In Pune Maharashtra,n/a,N/A,N/A,n/a,N/A,2024-04-02,None,None,...,None,None,None,None,None,None,None,None,None,None
208651,NaN,Information Security Associate Jobs In Bengaluru,n/a,N/A,N/A,n/a,N/A,2024-04-02,None,None,...,None,None,None,None,None,None,None,None,None,None
208712,NaN,Front Desk Executive / Receptionist (Female Ca...,technocruitx universal services,0 - 5 years,3 Lacs P.A.,"Mumbai,Bangalore",3 days ago,2024-04-02,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271822,NaN,Manager Sales - electronic components & Automo...,sr electro llp,5 - 8 years,"60,000-80,000 P.A.","Chennai,Chennai,Delhi",15 days ago,2024-04-04,None,Sales,...,None,None,None,None,None,None,None,None,None,None
271841,NaN,Blockchain Developer,synkcode fzco,2 - 5 years,"70,000-90,000 P.A.","Chennai,Bangalore,Ahmedabad,Chennai,Bangalore",10 days ago,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271864,NaN,Sales Executive Jobs In Hyderabad Secunderabad...,n/a,N/A,N/A,n/a,N/A,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None
271890,NaN,Maintenance Executive Jobs In Pune Maharashtra,n/a,N/A,N/A,n/a,N/A,2024-04-04,None,None,...,None,None,None,None,None,None,None,None,None,None


In [168]:
df = df[~df['salmin'].isna()]

In [169]:
df['ori_experience'] = df['experience']

C:\Users\PM\AppData\Local\Temp\ipykernel_13548\3958780136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ori_experience'] = df['experience']


In [170]:
wrong = []
for i in range(53):
    wrong.append(str(i)+' Yrs')
def correct_exp(exps):
    # print(exps)
    if pd.isna(exps) or exps == 'Nan' or exps == 'N/A':
        return np.nan
    exps = exps.replace('years','Yrs').replace('year','Yrs')
    if "+" in exps:
        return '15+'
    if ('-' not in exps) and ('Yrs' in exps):#for 0 year(s) of experience
        try:
            exps = exps.split('Yrs')[0].strip().split(' ')[-1] + ' Yrs'
        except:
            pass    
    wrong1 = ['entry level']
    if exps in wrong1:
        exps = '0-2'    
    
    if exps in wrong:
        num = int(exps.split(' ')[0])
        if num > 11:
            exps = '11-14'
        elif num > 8:
            exps = '8-11'
        elif num > 5:
            exps = '5-8'
        elif num > 2:
            exps = '2-5'
        else:
            exps = '0-2'

    if 'to' in exps: #From 5 to 10 year(s) of experience
        exp1 = exps.split('to')[0].strip().split(' ')[-1]
        exp2 = exps.split('to')[1].strip().split(' ')[0]
        exps = '-'.join([exp1,exp2])
    
    strang = int(exps[0])
    enrang = int(exps.split('-')[1].strip().split(' ')[0])
    # print(strang,'**',enrang)
    if strang >= 0 and enrang <= 3:
        return '0-2'
    elif strang >= 2 and enrang <= 5:
        return '2-5'
    elif strang >= 5 and enrang <= 8:
        return '5-8'
    elif strang >= 8 and enrang <= 11:
        return '8-11'
    elif strang >= 11 and enrang <= 14:
        return '11-14'
    elif strang > 15:
        return '15+'
    else:
        return exps

df['experience'] = df['experience'].apply(correct_exp)

C:\Users\PM\AppData\Local\Temp\ipykernel_13548\3201711351.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['experience'] = df['experience'].apply(correct_exp)


In [171]:
# df = df[~(df['experience'].isna()) | (df['experience'].str.contains('nan'))]

In [172]:
df['experience'].unique()

array(['5-8', '0-2', '8-11', nan, '2-5', '11-14', '15+', '0 - 4 Yrs',
       '0 - 10 Yrs', '6 - 11 Yrs', '1 - 4 Yrs', '0 - 5 Yrs', '4 - 6 Yrs',
       '4 - 8 Yrs', '3 - 8 Yrs', '0 - 8 Yrs', '6 - 10 Yrs', '2 - 6 Yrs',
       '2 - 7 Yrs', '3 - 7 Yrs', '1 - 6 Yrs', '7 - 12 Yrs', '2 - 8 Yrs',
       '3 - 6 Yrs', '2 - 12 Yrs', '5 - 10 Yrs', '4 - 9 Yrs', '5 - 9 Yrs',
       '6 - 9 Yrs', '1 - 5 Yrs', '3 - 10 Yrs', '10 - 15 Yrs', '7 - 9 Yrs',
       '8 - 12 Yrs', '12 - 16 Yrs', '10 - 20 Yrs', '13 - 17 Yrs',
       '7 - 11 Yrs', '15 - 18 Yrs', '12 - 15 Yrs', '10 - 14 Yrs',
       '8 - 13 Yrs', '4 - 7 Yrs', '15 - 19 Yrs', '9 - 14 Yrs',
       '20 - 25 Yrs', '2 - 9 Yrs', '7 - 10 Yrs', '3 - 9 Yrs',
       '4 - 10 Yrs', '12 - 14 Yrs', '9 - 12 Yrs', '0 - 7 Yrs',
       '10 - 12 Yrs', '11 - 16 Yrs', '0 - 20 Yrs', '2 - 10 Yrs',
       '15 - 17 Yrs', '1 - 7 Yrs', '0 - 6 Yrs', '2 - 18 Yrs',
       '20 - 30 Yrs', '0 - 9 Yrs', '5 - 12 Yrs', '11 - 14 Yrs',
       '3 - 15 Yrs', '3 - 12 Yrs', '2 - 15 Yrs', '

In [173]:
df['minexp'] = df['experience'].apply(lambda x: float(x.split('-')[0].strip()) if pd.notnull(x) and ('+' not in x)  else None)
df['maxexp'] = df['experience'].apply(lambda x: float(x.split('-')[1].strip().split(' ')[0]) if pd.notnull(x) and ('+' not in x) else None)
df['avg_experience'] = (df['minexp'] + df['maxexp'])//2

C:\Users\PM\AppData\Local\Temp\ipykernel_13548\1471534559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['minexp'] = df['experience'].apply(lambda x: float(x.split('-')[0].strip()) if pd.notnull(x) and ('+' not in x)  else None)
C:\Users\PM\AppData\Local\Temp\ipykernel_13548\1471534559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['maxexp'] = df['experience'].apply(lambda x: float(x.split('-')[1].strip().split(' ')[0]) if pd.notnull(x) and ('+' not in x) else None)
C:\Users\PM\AppData\Local

In [174]:
for title in df['mapped_job_title'].unique(): # create xp classes as exists.
    if 'unclassified' not in title:    
        titfiltdf = df[df['mapped_job_title'] == title] # filter df with it.
        exp_lis = [['15+',[15,50]],['11-14',[11,14]],['8-11',[8,11]],['5-8',[5,8]],['2-5',[2,5]],['0-2',[0,2]]]
        for exp in exp_lis:    
            salmin,salmax = titfiltdf[titfiltdf['experience'] == exp[0]][['salmin', 'salmax']].median()
            notfil = ~df['experience'].isin(['5-8', '0-2', '2-5', '8-11', '11-14', '15+'])
            titlefilt = df['mapped_job_title'] == title
            salfilt = (df['maxexp'] >= exp[1][0]) & (df['maxexp'] <= exp[1][1])
            if pd.notna(salmin) and pd.notna(salmax):
                # df.loc[notfil & titlefilt & salfilt,['salmin', 'salmax']] = salmin,salmax
                df.loc[notfil & titlefilt & salfilt,'experience'] = exp[0]
            else:
                # print('no',title,exp)
                df.loc[notfil & titlefilt & salfilt,'experience'] = exp[0]

In [175]:
df['salmin'].unique()

array([1.60e+01, 5.00e-01, 2.50e+00, 4.00e+00, 2.25e+00, 1.00e+01,
       2.00e+00, 3.50e+00, 1.00e+00, 8.00e-01, 1.75e+00, 1.80e+01,
       3.00e+00, 1.25e+00, 3.25e+00, 2.50e+01, 9.00e-01, 7.00e+00,
       1.50e+01, 1.70e+01, 1.20e+01, 8.50e+00, 1.10e+01, 5.00e+00,
       2.00e+01, 6.00e+00, 2.75e+00, 2.75e+01, 1.50e+00, 2.25e+01,
       4.00e+01, 3.50e+01, 9.00e+00, 8.00e+00, 3.75e+00, 6.00e-01,
       4.25e+00, 7.50e+00, 5.50e+00, 3.00e+01, 4.75e+00, 4.50e+00,
       1.40e+01, 3.25e+01, 6.50e+00, 3.75e+01, 7.00e+01, 5.00e+01,
       1.00e+02, 9.50e+00, 1.30e+01, 7.00e-01, 7.50e+01, 1.90e+01,
       4.50e+01, 6.00e+01, 5.50e+01, 8.00e+01, 6.50e+01, 6.50e+02,
       8.50e+01, 3.00e+02, 0.00e+00, 1.25e+02, 1.50e+02, 4.75e+02,
       3.25e+02, 2.75e+02, 1.75e+02, 4.00e+02])

In [176]:
df['experience'].unique()

array(['5-8', '0-2', '8-11', nan, '2-5', '11-14', '15+', '0 - 4 Yrs',
       '0 - 10 Yrs', '6 - 11 Yrs', '1 - 4 Yrs', '0 - 5 Yrs', '4 - 6 Yrs',
       '4 - 8 Yrs', '3 - 8 Yrs', '0 - 8 Yrs', '6 - 10 Yrs', '2 - 6 Yrs',
       '2 - 7 Yrs', '3 - 7 Yrs', '1 - 6 Yrs', '7 - 12 Yrs', '2 - 8 Yrs',
       '3 - 6 Yrs', '2 - 12 Yrs', '5 - 10 Yrs', '4 - 9 Yrs', '5 - 9 Yrs',
       '6 - 9 Yrs', '1 - 5 Yrs', '3 - 10 Yrs', '10 - 15 Yrs', '7 - 9 Yrs',
       '8 - 12 Yrs', '12 - 16 Yrs', '10 - 20 Yrs', '13 - 17 Yrs',
       '7 - 11 Yrs', '15 - 18 Yrs', '12 - 15 Yrs', '10 - 14 Yrs',
       '8 - 13 Yrs', '4 - 7 Yrs', '15 - 19 Yrs', '9 - 14 Yrs',
       '20 - 25 Yrs', '2 - 9 Yrs', '7 - 10 Yrs', '3 - 9 Yrs',
       '4 - 10 Yrs', '12 - 14 Yrs', '9 - 12 Yrs', '0 - 7 Yrs',
       '10 - 12 Yrs', '11 - 16 Yrs', '0 - 20 Yrs', '2 - 10 Yrs',
       '15 - 17 Yrs', '1 - 7 Yrs', '0 - 6 Yrs', '2 - 18 Yrs',
       '20 - 30 Yrs', '0 - 9 Yrs', '5 - 12 Yrs', '11 - 14 Yrs',
       '3 - 15 Yrs', '3 - 12 Yrs', '2 - 15 Yrs', '

In [177]:
df[df['experience'].isin(['15+','11-14'])]

,id,job_title,company_name,experience,salary,location,posted_on,extracted_on,skill1,skill2,...,skill31,skill32,skill33,skill34,skill35,skill36,skill37,skill38,skill39,skill40
23,24.0,Lead Software Engineer/Staff software Backend ...,amolitalents,11-14,25-40 Lacs PA,Chennai,1 Day Ago,2023-08-22,Kubernetes,Node.js,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,32.0,Sap Sd Consultant,wipro,11-14,15-30 Lacs PA,"Hyderabad,Chennai,Chennai",1 Day Ago,2023-08-22,SAP Sales and Distribution (SAP SD),SAP Sales and Distribution (SAP SD),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,44.0,Servicenow Developer,sharp gaze tech services,11-14,11-21 Lacs PA,Chennai,Just Now,2023-08-22,Servicenow,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,46.0,Netconf Server Architect,amantya technologies,11-14,20-35 Lacs PA,"Chennai,Bangalore",1 Day Ago,2023-08-22,Python,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,60.0,Scrum Master,symphoni hr,11-14,27.5-42.5 Lacs PA,Chennai,1 Day Ago,2023-08-22,None,Cash Flow Management,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271063,NaN,ServiceNow Development Manager,orange business services,11-14,0-3,"Gurugram,haryana",1 day ago,2024-04-04,None,Servicenow,...,None,None,None,None,None,None,None,None,None,None
271721,NaN,Security Specialist,team computers,11-14,0-3,"Chennai,Delhi,Chennai",6 days ago,2024-04-04,Cyber Security,None,...,None,None,None,None,None,None,None,None,None,None
271828,NaN,SAP ABAP HANA Consultant,enterprise infionics (i),11-14,0-3,"Hyderabad,telangana,Chennai,Bangalore,Chennai,...",17 days ago,2024-04-04,None,SAP ABAP,...,None,None,None,None,None,None,None,None,None,None
271848,NaN,A Leading MNC hiring Servicenow Architect,saaki argus and averil consulting,15+,0-3,"Chennai,Chennai,Chennai",16 days ago,2024-04-04,Servicenow,None,...,None,None,None,None,None,None,None,None,None,None


In [178]:
for i in topskillsdf.columns:
    if 'acc' in i:
        print(i)

In [179]:
df['mapped_average_sal'] = (df['salmin'] + df['salmax'])/2

C:\Users\PM\AppData\Local\Temp\ipykernel_13548\396240511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mapped_average_sal'] = (df['salmin'] + df['salmax'])/2


In [180]:
df.columns

Index(['id', 'job_title', 'company_name', 'experience', 'salary', 'location',
       'posted_on', 'extracted_on', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'role', 'industry_type',
       'department', 'employment_type', 'role_category', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills', 'salmin',
       'salmax', 'ori_experience', 'minexp', 'maxexp', 'title_id',
       'avg_experience', 'description', 'link', 'sal_range', 'skill9',
       'skill10', 'skill11', 'skill12', 'skill13', 'skill14', 'skill15',
       'skill16', 'skill17', 'skill18', 'skill19', 'skill20', 'skill21',
       'skill22', 'skill23', 'skill24', 'skill25', 'skill26', 'skill27',
       'skill28', 'skill29', 'skill30', 'skill31', 'skill32', 'skill33',
       'skill34', 'skill35', 'skill36', 'skill37', 'skill38', 'skill39',
       'skill40'],
      dtype='object')

In [181]:
df = df[~( (df['maxexp'] < 10) & (df['salmin'] > 100) )]

In [182]:
df[df['salmin'] > 100][['experience','ori_experience','salary','salmin','salmax','mapped_job_title','job_title']]

,experience,ori_experience,salary,salmin,salmax,mapped_job_title,job_title
83670,NaN,None,6.5-7.5 Cr PA,650.0,750.0,unclassified,Premier Business Sales Manager Chief Business ...
136484,15+,15+,3-5 Cr P.A.,300.0,500.0,unclassified,Job Opening For Senior Business Execution Manager
198653,18 - 28 Yrs,18 - 28 Yrs,1.25-1.75 Cr P.A.,125.0,175.0,unclassified,CFO For a leading MNC manufacturing org. in Hy...
198667,20 - 26 Yrs,20 - 26 Yrs,1.5-2 Cr P.A.,150.0,200.0,unclassified,Global Insurance Head - SaaS
198668,15+,15+,1.5-2.25 Cr P.A.,150.0,225.0,"BFSI, Investments & Trading",Head of Global Banking
262086,18 - 28 Yrs,18 - 28 years,1.25-1.75 Cr P.A.,125.0,175.0,unclassified,CFO For a leading MNC manufacturing org. in Hy...
262101,20 - 26 Yrs,20 - 26 years,1.5-2 Cr P.A.,150.0,200.0,unclassified,Global Insurance Head - SaaS
262103,15+,20 - 27 years,1.5-2.25 Cr P.A.,150.0,225.0,"BFSI, Investments & Trading",Head of Global Banking


In [183]:
df.columns

Index(['id', 'job_title', 'company_name', 'experience', 'salary', 'location',
       'posted_on', 'extracted_on', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'role', 'industry_type',
       'department', 'employment_type', 'role_category', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills', 'salmin',
       'salmax', 'ori_experience', 'minexp', 'maxexp', 'title_id',
       'avg_experience', 'description', 'link', 'sal_range', 'skill9',
       'skill10', 'skill11', 'skill12', 'skill13', 'skill14', 'skill15',
       'skill16', 'skill17', 'skill18', 'skill19', 'skill20', 'skill21',
       'skill22', 'skill23', 'skill24', 'skill25', 'skill26', 'skill27',
       'skill28', 'skill29', 'skill30', 'skill31', 'skill32', 'skill33',
       'skill34', 'skill35', 'skill36', 'skill37', 'skill38', 'skill39',
       'skill40'],
      dtype='object')

In [184]:
# topskillsdf.to_excel('role_wise_top_skills.xlsx',index=False)

In [185]:
skillsdf = pd.DataFrame.from_dict(renjidict, orient='index').T

In [186]:
price_a_job_relavent_skills_dict = []
for col in skillsdf.columns:
    for skill in skillsdf[col].values:
        if isinstance(skill, str):
            price_a_job_relavent_skills_dict.append({'skill':skill, 'title':col})

price_a_job_relavent_skills = pd.DataFrame(price_a_job_relavent_skills_dict)
price_a_job_relavent_skills = price_a_job_relavent_skills.sort_values('skill').reset_index(drop=True)
price_a_job_relavent_skills['id'] = price_a_job_relavent_skills.index + 1
price_a_job_relavent_skills = price_a_job_relavent_skills.reindex(columns=['id','title','skill'])
price_a_job_relavent_skills = price_a_job_relavent_skills.rename(columns={'title':'title_id'})
price_a_job_relavent_skills['title_id'] = price_a_job_relavent_skills['title_id'].replace(price_a_job_titles_dict)

In [187]:
skillsdf.columns = skillsdf.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()
# skillsdf.to_excel('skills_and_relevant_roles.xlsx',index =False)

In [188]:
# topskillsdf = pd.read_excel('role_wise_top_skills.xlsx')
topskillsdf.columns = topskillsdf.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()

In [189]:
df.columns = df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()
df.columns

Index(['id', 'job_title', 'company_name', 'experience', 'salary', 'location',
       'posted_on', 'extracted_on', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'role', 'industry_type',
       'department', 'employment_type', 'role_category', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills', 'salmin',
       'salmax', 'ori_experience', 'minexp', 'maxexp', 'title_id',
       'avg_experience', 'description', 'link', 'sal_range', 'skill9',
       'skill10', 'skill11', 'skill12', 'skill13', 'skill14', 'skill15',
       'skill16', 'skill17', 'skill18', 'skill19', 'skill20', 'skill21',
       'skill22', 'skill23', 'skill24', 'skill25', 'skill26', 'skill27',
       'skill28', 'skill29', 'skill30', 'skill31', 'skill32', 'skill33',
       'skill34', 'skill35', 'skill36', 'skill37', 'skill38', 'skill39',
       'skill40'],
      dtype='object')

In [190]:
topskillsdf.columns[-2]

'medical_biller'

In [191]:
# df.to_excel('salary_and_company_mapping.xlsx',index = False)

In [192]:
# while True:
#     check = topskillsdf.columns[-2]
#     try:
#         for i in topskillsdf.columns:
#             if ('_count' in i):
#                 del topskillsdf[i]
#     except:
#         continue
#     if check in i:
#         break

In [193]:
df.reset_index(drop=True,inplace=True)
df['id'] = df.index + 1

In [194]:
# df= pd.read_excel('salary_and_company_mapping.xlsx')

In [195]:
df.columns

Index(['id', 'job_title', 'company_name', 'experience', 'salary', 'location',
       'posted_on', 'extracted_on', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'role', 'industry_type',
       'department', 'employment_type', 'role_category', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills', 'salmin',
       'salmax', 'ori_experience', 'minexp', 'maxexp', 'title_id',
       'avg_experience', 'description', 'link', 'sal_range', 'skill9',
       'skill10', 'skill11', 'skill12', 'skill13', 'skill14', 'skill15',
       'skill16', 'skill17', 'skill18', 'skill19', 'skill20', 'skill21',
       'skill22', 'skill23', 'skill24', 'skill25', 'skill26', 'skill27',
       'skill28', 'skill29', 'skill30', 'skill31', 'skill32', 'skill33',
       'skill34', 'skill35', 'skill36', 'skill37', 'skill38', 'skill39',
       'skill40'],
      dtype='object')

In [196]:
# del df['job_description']
del df['description']

In [197]:
df.rename(columns={'current_date':'extracted_on'},inplace=True)

In [198]:
df = df.reindex(columns=['id','job_title', 'company_name', 'experience', 'salary', 'location',
       'posted_on', 'extracted_on', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'role', 'industry_type',
       'department', 'employment_type', 'role_category', 'mapped_average_sal',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills', 'salmin', 'salmax', 'ori_experience', 'minexp', 'maxexp',
       'title_id','avg_experience'])

In [199]:
len(df['company_name'].unique())

29154

In [200]:
df['title_id'].unique()

array([22,  0, 16, 21, 72,  6, 60, 13, 79, 42, 66, 81, 40, 69, 31, 76, 17,
       35, 44, 20, 32, 54, 57, 77, 75, 18,  8, 25, 55, 26,  7, 45, 39, 51,
       34, 80, 63, 67, 59, 74, 15, 58, 68, 48, 78, 19, 10, 49,  2, 29, 64,
       38, 33, 27,  9,  3, 71, 14, 50, 46,  1, 23, 11, 30, 61, 12, 43, 47,
       28, 37, 53, 73,  5, 62, 36, 56,  4, 24, 70, 41, 52, 65],
      dtype=int64)

In [201]:
df['industry_type'] = df['industry_type'].fillna('unclassified')

In [202]:
df.to_parquet('latest_backup_mixed.parquet',index=False)

chunk_size = 10000  # Adjust the chunk size as needed
num_chunks = len(df) // chunk_size + 1

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

connection = mysql.connector.connect(
host=host,
user=user,
password=password,
database=database
)

cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS paypulse_profiles")
connection.commit()

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    chunk_df = df.iloc[start_idx:end_idx]
    chunk_df.to_sql(name='paypulse_profiles', con=engine, if_exists='append', index=False)
    print(f'\rprogress: {i+1},{num_chunks}',end=' ')
print("Data inserted successfully.")

progress: 28,28 Data inserted successfully.


In [203]:
# cursor = connection.cursor()
# cursor.execute("DROP TABLE IF EXISTS naukri_extract")
# connection.commit()
# cursor = connection.cursor()
# cursor.execute("DROP TABLE IF EXISTS skills_and_relevant_roles")
# connection.commit()
# cursor = connection.cursor()
# cursor.execute("DROP TABLE IF EXISTS topskills")
# connection.commit()


In [204]:
price_a_job_relavent_skills.to_sql(name='paypulse_relevant_skills', con=engine, if_exists='replace', index=False)

10221

In [205]:
price_a_job_topskills.to_sql(name='paypulse_topskills', con=engine, if_exists='replace', index=False)

1620

In [206]:
price_a_job_titles.to_sql(name='paypulse_titles', con=engine, if_exists='replace', index=False)

81

In [207]:
# engine.dispose()

In [208]:
# cursor.execute("select * from secondstorey.tracking_data_3")
# rows = cursor.fetchall()
# columns = [desc[0] for desc in cursor.description]
# data_frame = pd.DataFrame(rows, columns=columns)
# data_frame

In [209]:
# import requests

# def get_ip_location(ip_address):
#     access_token = '255ebfcc46428e'  # Replace with your IPinfo API access token
#     api_url = f'https://ipinfo.io/{ip_address}/json?token={access_token}'
    
#     response = requests.get(api_url)
#     data = response.json()
    
#     return data

# def get_location(row):
#     ip_address = row
#     location_info = get_ip_location(ip_address)
#     return location_info

# data_frame['Location'] = data_frame['path'].apply(get_location)

# cities = []
# regions = []

# for entry in data_frame['Location']:
#     if 'city' in entry:
#         cities.append(entry['city'])
#     else:
#         cities.append(None) 
        
#     if 'region' in entry:
#         regions.append(entry['region'])
#     else:
#         regions.append(None)

# data_frame['City'] = cities
# data_frame['Region'] = regions


In [210]:
# data_frame.to_excel('tracking_data_3.xlsx',index=False)